# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tqdm

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import numpy as np 
import pandas as pd 
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os  
import re
import json
import seaborn as sb
import nltk
import glob
import spacy
from tqdm import tqdm
from pathlib import Path
from copy import deepcopy
import numba
from numba import prange
from multiprocessing import Pool, cpu_count
from nltk.corpus import stopwords
from gensim.utils import deaccent



In [ ]:

import functools
import re
from IPython.core.display import display, HTML
import string

In [ ]:
pip install pip-autoremove

In [ ]:
pip autoremove pandas -y

ERROR: unknown command "autoremove"


In [ ]:
pip install pandas

In [ ]:
import pandas as pd

In [ ]:
metadata = pd.read_csv('/content/drive/MyDrive/NEW TASK_REVIEW3/metadata.csv', dtype={
    'pubmed_id': str,
    'pmcid': str,
    'doi' : str,
    'mag_id' : str,
    'arxiv_id': str,
    's2_id': str,
    'who_covidence_id': str,
    'pmc_json_files': str
})

In [ ]:
num_entries = len(metadata)
print("There are {} many entries in this dataset:".format(num_entries))

metadata_with_text = metadata[metadata['pdf_json_files'].isna() == False]
with_full_text = len(metadata_with_text)
print("-- {} have full text entries".format(with_full_text))

with_doi = metadata['doi'].count()
print("-- {} have DOIs".format(with_doi))

with_pmcid = metadata['pmcid'].count()
print("-- {} have PubMed Central (PMC) ids".format(with_pmcid))

with_microsoft_id = metadata['mag_id'].count()
print("-- {} have Microsoft Academic paper ids".format(with_microsoft_id))

There are 301667 many entries in this dataset:
-- 111985 have full text entries
-- 186832 have DOIs
-- 116337 have PubMed Central (PMC) ids
-- 0 have Microsoft Academic paper ids


In [ ]:
data_path = '/content/drive/MyDrive/NEW TASK_REVIEW3/'

In [ ]:
metadata.drop(['mag_id'],axis=1,inplace=True)
metadata.head(3)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [ ]:
class DataHandler(object):
    """
    Class that is used to load all the data from the CORD-19 dataset
    """
    def __init__(self, datapath):
        self.datapath = datapath

    def read_papers(self, filelist):
        """
        Read json files from list of files
        """
        list_json_data = []

        for filename in filelist:
            file = json.load(open(filename, "rb"))
            list_json_data.append(file)

        return list_json_data

    def read_files_paths(self, folderpath, fileext=".json"):
        """
        Get paths to all the files inside a folder recursively. Set an extensions if needed
        """
        fls = [
            os.path.join(root, fn)
            for root, dirs, files in os.walk(Path(folderpath))
            for fn in files
            if Path(fn).suffix == fileext
        ]

        return fls

    def parse_authors(self, authors):
        """
        Parse authors field
        """
        authors_ls = []
        names_ls = []
        affiliations_ls = []
        emails_ls = []

        for author in authors:

            author_text = []

            # Parse name
            middle_name = " ".join(author["middle"])
            full_name = (
                " ".join([author["first"], middle_name, author["last"]])
                if author["middle"]
                else " ".join([author["first"], author["last"]])
            )
            author_text.append(full_name)
            names_ls.append(full_name)

            # Parse affiliation
            affiliation = author["affiliation"]
            if affiliation:
                affiliation_text = []
                laboratory = affiliation["laboratory"]
                institution = affiliation["institution"]
                location = affiliation["location"]
                if laboratory:
                    affiliation_text.append(laboratory)
                if institution:
                    affiliation_text.append(institution)
                if location:
                    affiliation_text.append(" ".join(list(location.values())))

                affiliation_text = ", ".join(affiliation_text)
                author_text.append(f"({affiliation_text})")
                affiliations_ls.append(affiliation_text)

            # Parse email
            email = author["email"]
            if email:
                author_text.append(f"[{email}]")
                emails_ls.append(email)

            # Concat info
            author_text = ", ".join(author_text)
            authors_ls.append(author_text)

        return {
            "authors_full": "; ".join(authors_ls),
            "names": "; ".join(names_ls),
            "emails": "; ".join(emails_ls),
            "affiliations": "; ".join(affiliations_ls),
        }

    def parse_abstract_body(self, body_text, raw_mode=False):
        """
        Parse abstract and body fields
        """
        entries_ls = []

        for entry in body_text:
            # Get section and its text
            text = entry["text"]
            section = entry["section"]

            if raw_mode != True:
                entries_ls.append(section)
                entries_ls.append(text)
            else:
                entries_ls.append({"text": text, "section": section})

        if raw_mode != True:
            return "\n\n".join(entries_ls)
        else:
            return entries_ls

    def parse_bib(self, bibs):
        """
        Parse bibliography field
        """
        if type(bibs) == dict:
            bibs = list(bibs.values())
        bibs = deepcopy(bibs)

        bibs_clean = []
        for bib in bibs:
            title = bib["title"]
            authors = bib["authors"]
            year = bib["year"]
            venue = bib["venue"]
            bibs_clean.append(f"{title} [{year}, {venue}]")

        return "; ".join(bibs_clean)

    def build_df(self, subfolder_papers, no_full_text=False):
        """
        Build a final Dataframe
        """
        list_json = self.read_files_paths(self.datapath + subfolder_papers)
        list_json_data = self.read_papers(list_json)

        raw_data = {
            "paper_id": [],
            "title": [],
            "authors": [],
            "authors_names": [],
            "authors_affiliations": [],
            "authors_emails": [],
            "abstract": [],
            "text": [],
            "bibliography": [],
            "raw_bibliography": [],
        }
        for file in tqdm(list_json_data):
            
            raw_data["paper_id"].append(file["paper_id"])
            raw_data["title"].append(file["metadata"]["title"])

            authors = self.parse_authors(file["metadata"]["authors"])
            raw_data["authors"].append(authors["authors_full"])
            raw_data["authors_names"].append(authors["names"])
            raw_data["authors_affiliations"].append(authors["affiliations"])
            raw_data["authors_emails"].append(authors["emails"])
            
            if "abstract" in file:
                raw_data["abstract"].append(self.parse_abstract_body(file["abstract"]))
            else:
                raw_data["abstract"].append("")
            if no_full_text == True:
                raw_data["text"].append("")
            else:
                raw_data["text"].append(self.parse_abstract_body(file["body_text"]))
            raw_data["bibliography"].append(self.parse_bib(file["bib_entries"]))
            raw_data["raw_bibliography"].append(file["bib_entries"])

        df_data = pd.DataFrame(raw_data)

        return df_data

In [ ]:
datahandler = DataHandler(data_path)

In [ ]:
df_json = datahandler.build_df('jsonfiles10k')
df_json['subset'] = "json"

100%|██████████| 9987/9987 [00:16<00:00, 596.08it/s]


In [ ]:
len(df_json)

9987

In [ ]:
#  drop duplicates according to paper_id and abstract
df_json = df_json.drop_duplicates(subset=['paper_id', 'abstract',]).reset_index(drop=True)

In [ ]:
columns_to_keep = [
    "paper_id",
    "title_x",
    "authors_x",
    "abstract_x",
    "publish_time",
    "text",
    "bibliography",
    "raw_bibliography",
    "subset",
    "doi"
]

In [ ]:
# Merge raw data with metadata
df_merged = pd.merge(metadata, df_json, left_on='sha', right_on='paper_id', how='inner')[columns_to_keep]

In [ ]:
df_merged = df_merged.rename(columns={'source_x': 'source', 'title_x': 'title', 'authors_x': 'authors', 'abstract_x': 'abstract',})

In [ ]:
df_merged.columns

Index(['paper_id', 'title', 'authors', 'abstract', 'publish_time', 'text',
       'bibliography', 'raw_bibliography', 'subset', 'doi'],
      dtype='object')

In [ ]:
# Replace empty string fields with nans
df_merged = df_merged.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# Drop duplicates again and reset index
df_merged.drop_duplicates(subset='abstract', inplace=True)
df_merged.reset_index(drop=True, inplace=True)

In [ ]:
# Create column with all text data
df_merged['title_abstract_text'] = df_merged['title'].astype(str) + "\n\n" + df_merged['abstract'].astype(str) + "\n\n" + df_merged['text'].astype(str)

In [ ]:
# Check for nans
df_merged.loc[:, df_merged.isnull().any()].columns

Index(['authors', 'abstract', 'doi'], dtype='object')

In [ ]:
# Filling Nan's
df_merged['authors'].fillna("", inplace=True)
df_merged['abstract'].fillna("", inplace=True)

In [ ]:
df=df_merged

In [ ]:
#keep only documents with covid,corona...
def search_focus(df):
    dfa = df[df['text'].str.contains('covid')]
    dfb = df[df['text'].str.contains('sars')]
    dfc = df[df['text'].str.contains('cov.*\d',regex=True)]
    dfd = df[df['text'].str.contains('ncov')]
    dfe = df[df['text'].str.contains('corona')]
    dfg = df[df['text'].str.contains('mars')]
    dfh = df[df['text'].str.contains('wuhan.*pneumonia',regex=True)]
    dfi = df[df['text'].str.contains('novel.*corona',regex=True)]
    dfj=   df[df['text'].str.contains('wuhan.*virus',regex=True)]
    dfk=  df[df['text'].str.contains('diagnostics',regex=True)]


    frames=[dfa,dfb,dfc,dfd,dfe,dfg,dfh,dfi,dfj,dfk]
    #frames=[dfa,dfb,dfc,dfd]
    df = pd.concat(frames)

    df=df.drop_duplicates(subset='title', keep="first")
    return df

#drop duplicate titles
df = df.drop_duplicates(subset='title', keep="first")
#keep only 2020 dated papers
df=df[df['publish_time'].str.contains('2020')]
#show 5 lines of the new dataframe
df=search_focus(df)
print (df.shape)
df.head()

(3212, 11)


,paper_id,title,authors,abstract,publish_time,text,bibliography,raw_bibliography,subset,doi,title_abstract_text
1134,e97c5b77a1087083a18fe6dccdf2f7029cfc8ad7,Emergence and Reemergence of Severe Acute Resp...,"Baxi, Preeti; Saxena, Shailendra K.","The positive-strand RNA viruses, severe acute ...",2020-04-30,Introduction\n\nCoronavirus belongs to the lar...,"COVID-19 infection: origin, transmission, and ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'COVID-1...",json,10.1007/978-981-15-4814-7_13,Emergence and Reemergence of Severe Acute Resp...
1145,ec7447ab5e1341c5e5818a5dd302caeb51bae7ca,From Diversity to Coordination: A European App...,"PACCES, Alessio M.; WEIMER, Maria",The COVID-19 pandemic is changing the face of ...,2020-04-16,\n\nThe COVID-19 pandemic is rapidly changing ...,Health Security Committee summary reports from...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'Health ...",json,10.1017/err.2020.36,From Diversity to Coordination: A European App...
1172,f2ec76273570c1909ae680e1bc2ea0c12480fd21,Recomendaciones De La Sociedad Española De Cir...,"Manuel Garrido Jiménez, José; Aroca, Jose Migu...","In the actual COVID-19 Pandemic, the Spanish S...",2020-05-18,RECOMENDACIONES DE LA SOCIEDAD ESPAÑOLA DE CIR...,Supporting the Health Care Workforce During th...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'Support...",json,10.1016/j.circv.2020.04.002,Recomendaciones De La Sociedad Española De Cir...
1324,ef95bda8107b3bbd8fa5e4e2f9b89e6c2d4ade86,Policing During the Time of Corona: The Indian...,"Biswas, Debajyoti; Sultana, Parvin",According to Oxford COVID-19 Government Respon...,2020-05-31,Introduction\n\nAccording to Oxford COVID-19 G...,The Role of Religious Leaders in Promoting Hea...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'The Rol...",json,10.1093/police/paaa024,Policing During the Time of Corona: The Indian...
1456,e1411444491ad66fb32ede92081dd99eea2b0272,"Fighting impunity in hate crime — history, eth...","Bhat, M Mohsin Alam",This interview with writer and activist Harsh ...,2020-08-20,"\n\ninclusive Constitution. 6 Within weeks, an...",The first major communal riot in post-colonial...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'The fir...",json,10.1007/s41020-020-00118-1,"Fighting impunity in hate crime — history, eth..."


In [ ]:
corpus=df

In [ ]:
def clean(col):
    col = col.replace('\n', '')
    col = col.replace('\r', '')    
    col = col.replace('\t', '')
    col = re.sub("\[[0-9]+(,[0-9]+)*\]", "", col)
    col = re.sub("\([0-9]+(,[0-9]+)*\)", "", col)
    col = re.sub("\{[0-9]+(,[0-9]+)*\}", "", col)

    return col
    
corpus['abstract'] = corpus['abstract'].apply(clean)
corpus['text'] = corpus['text'].apply(clean)

In [ ]:
corpus.rename(columns={'text':'body_text'}, inplace=True)
corpus['text'] = corpus.title + ' ' + corpus.abstract + ' ' + corpus.body_text

In [ ]:
# Drop abstract and body text columns
corpus = corpus.drop(['body_text', 'abstract','title_abstract_text'], axis=1)

In [ ]:
pip install langdetect

     |████████████████████████████████| 983kB 6.5MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=0fd038c922973412071552dad29fea2199646b1e9b93034c50d9c73e23c03bc6
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
#Detect language of docs in corpus

from tqdm import tqdm
from langdetect import detect
from langdetect import DetectorFactory

# set seed
DetectorFactory.seed = 0

# hold label - language
languages = []

df=corpus
# go through each text
for ii in tqdm(range(0,len(df))):
    # split by space into list, take the first x intex, join with space
    text = df.iloc[ii]['text'].split(" ")
    
    title= df.iloc[ii]['title'].split(" ")
    
    try:
        if len(title) > 100:
            lang = detect(" ".join(title[:100]))
        elif len(title) > 0:
            lang = detect(" ".join(title[:len(title)]))
    # ught... beginning of the document was not in a good format
    except Exception as e:
        all_words = set(title)
        try:
            lang = detect(" ".join(all_words))
        except Exception as e:
            
            try:
                # let's try to label it through the text then
                lang = detect(df.iloc[ii]['text'])
            except Exception as e:
                lang = "unknown"
                
                pass
    
    # get the language
    #df['language']=lang
    languages.append(lang)

df['language']=languages

100%|██████████| 3212/3212 [00:17<00:00, 187.82it/s]


In [ ]:
from pprint import pprint

languages_dict = {}
for lang in set(languages):
    languages_dict[lang] = languages.count(lang)
    
print("Total: {}\n".format(len(languages)))
pprint(languages_dict)

Total: 3212

{'af': 1,
 'ca': 5,
 'de': 12,
 'en': 3126,
 'es': 22,
 'et': 1,
 'fr': 18,
 'hu': 1,
 'it': 11,
 'ko': 1,
 'lt': 1,
 'nl': 3,
 'pt': 2,
 'ro': 4,
 'tl': 3,
 'vi': 1}


In [ ]:
a=df[df['language']=='en']
len(a)

3126

In [ ]:
#keep only english
df = df[df['language'] == 'en'] 
len(df)

3126

In [ ]:
import nltk
nltk.download('punkt')

columns=list(df.columns)
columns.append('tokenized_text')

tokenized_df=pd.DataFrame(df,columns=columns)

word_tokens=[]
for index,row in df.iterrows():
  word_tokens.append(nltk.word_tokenize(row['text']))

df['tokenized_text']=word_tokens

df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,paper_id,title,authors,publish_time,bibliography,raw_bibliography,subset,doi,text,language,tokenized_text
1134,e97c5b77a1087083a18fe6dccdf2f7029cfc8ad7,Emergence and Reemergence of Severe Acute Resp...,"Baxi, Preeti; Saxena, Shailendra K.",2020-04-30,"COVID-19 infection: origin, transmission, and ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'COVID-1...",json,10.1007/978-981-15-4814-7_13,Emergence and Reemergence of Severe Acute Resp...,en,"[Emergence, and, Reemergence, of, Severe, Acut..."
1145,ec7447ab5e1341c5e5818a5dd302caeb51bae7ca,From Diversity to Coordination: A European App...,"PACCES, Alessio M.; WEIMER, Maria",2020-04-16,Health Security Committee summary reports from...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'Health ...",json,10.1017/err.2020.36,From Diversity to Coordination: A European App...,en,"[From, Diversity, to, Coordination, :, A, Euro..."
1324,ef95bda8107b3bbd8fa5e4e2f9b89e6c2d4ade86,Policing During the Time of Corona: The Indian...,"Biswas, Debajyoti; Sultana, Parvin",2020-05-31,The Role of Religious Leaders in Promoting Hea...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'The Rol...",json,10.1093/police/paaa024,Policing During the Time of Corona: The Indian...,en,"[Policing, During, the, Time, of, Corona, :, T..."
1456,e1411444491ad66fb32ede92081dd99eea2b0272,"Fighting impunity in hate crime — history, eth...","Bhat, M Mohsin Alam",2020-08-20,The first major communal riot in post-colonial...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'The fir...",json,10.1007/s41020-020-00118-1,"Fighting impunity in hate crime — history, eth...",en,"[Fighting, impunity, in, hate, crime, —, histo..."
1491,73f3955e4f65729a5fcee70f95d85cb4c8aac524,Why do so many trials of vitamin D supplementa...,"Boucher, Barbara J",2020-08-11,Rickets and the crippled child: an historical ...,"{'BIBREF0': {'ref_id': 'b0', 'title': 'Rickets...",json,10.1530/ec-20-0274,Why do so many trials of vitamin D supplementa...,en,"[Why, do, so, many, trials, of, vitamin, D, su..."


In [ ]:
queries=[

'What do we know about diagnostics and coronavirus?',
'New advances in diagnosing SARS-COV-2',
'Development of a point-of-care test and rapid bed-side tests',
'Diagnosing SARS-COV-2 with Nucleic-acid based tech',
'Diagnosing SARS-COV-2 with antibodies',
'How does viral load relate to disease presentations and likelihood of a positive diagnostic test?'         
]

# Ranking using BM25Okapi

In [ ]:
!pip install rank_bm25

from rank_bm25 import BM25Okapi
bm25=BM25Okapi(word_tokens)

In [ ]:
top_10_scores=[]
for query in queries:
  top_10_scores.append(bm25.get_scores(nltk.word_tokenize(query)))

#print(top_10_scores)
top10_scores_sorted=[]

for scores in top_10_scores:
  mylist=sorted(scores,reverse=True)
  #print(scores)
  top10_scores_sorted.append(mylist[:10])

print(top10_scores_sorted)
#bm25.get_top_n(tokenized_query, corpus, n=1)


[[27.669937339902987, 25.25086167392569, 24.54735500066335, 24.44283933397925, 23.811713284451212, 23.7999863148015, 23.66166359146295, 23.597626883446367, 23.435152167733794, 23.256059483360666], [16.425395973187033, 14.450642561179162, 14.263008594896846, 14.193658767767134, 13.945083837079835, 13.118135497301594, 13.046301128050601, 12.927674364909894, 12.694453117775138, 12.565777452730725], [29.398443532949866, 28.82496349639111, 28.722656102736448, 28.34884403824043, 27.70565277719211, 27.580749413280312, 27.29313439470707, 27.002348688828615, 26.99358448178587, 26.961999962319442], [20.211438416418538, 17.507062154076465, 15.97441992471769, 15.477544853755514, 14.83287504407695, 14.828693287481236, 14.738156197966639, 14.723498145705229, 14.54900175820548, 14.41082779422467], [13.663271321564022, 13.238073978115787, 13.235496862686151, 13.223823053714984, 12.868181584210324, 12.816554466324853, 12.80259416907678, 12.800850060744828, 12.458333477348457, 12.405272027329799], [42.4

In [ ]:
df_dict={}
for index,row in df.iterrows():
  df_dict[row['paper_id']]=pd.DataFrame(row)

#a=df_dict

In [ ]:
doc_list=[]
mystr_list=[]

for query in queries:
  doc_list.append(bm25.get_top_n(nltk.word_tokenize(query), list(df['paper_id']), n=10))

#doc_list


In [ ]:
from IPython.core.display import display, HTML

pd.set_option('display.max_colwidth',None)
m=0
for i in range(0,len(doc_list)):
  finaldf=pd.DataFrame(columns=['paper_id','title','doi','text','score'])
  display(HTML('<h1> '+queries[i]+'</h1>'))
  k=0
  for id in doc_list[i]:
    for index,row in df_dict[id].T.iterrows():
      link=str(row['doi'])
      title=row["title"]
      linka='http://doi.org/'+link
      final_link=linka
      to_append = [row['paper_id'],title,final_link,row['text'][:10],top10_scores_sorted[i][k]]
      df_length = len(finaldf)
      finaldf.loc[df_length] = to_append
      k+=1
  m+=1
  display(finaldf)

,paper_id,title,doi,text,score
0,e86a2d96cb6bea7787c58496b384c59b788017dd,COVID-19: Global Health Equity in Pandemic Response,http://doi.org/10.4269/ajtmh.20-0260,COVID-19:,27.669937
1,e54cfdf6d59ae0174571e71ead0061a9b32b80c3,PPE or not PPE - that is the question,http://doi.org/10.1038/s41415-020-1639-y,PPE or not,25.250862
2,eb819f89014a6e662d0208c962155b91e91d6584,Singing Away the Social Distancing Blues: Art Therapy in a Time of Coronavirus,http://doi.org/10.1177/0022167820927807,Singing Aw,24.547355
3,ec59134c9675007a1de94a4fcce28c139fb1baeb,"The Mortal Coil of Covid-19, Fake News, and Negative Epistemic Postdigital Inculcation",http://doi.org/10.1007/s42438-020-00192-7,The Mortal,24.442839
4,f8c120006ddc1c256306a35759e00aaa505b9c6b,Coronavirus Pandemic as Black Swan Event,http://doi.org/10.1007/978-3-030-49264-9_32,Coronaviru,23.811713
5,f4c762289a704d61cde8da3ddae21fc7a0ed5e99,Psychologist Self-Care During the Pandemic: Now More Than Ever,http://doi.org/10.1007/s42843-020-00010-5,Psychologi,23.799986
6,eef47e7531ebe9031aa62bd738e7cb36e9aade5e,Systematic review of international guidelines for tracheostomy in COVID-19 patients,http://doi.org/10.1016/j.oraloncology.2020.104844,Systematic,23.661664
7,eb98e3dd727cf16112cc9ed441c3a2105a332830,Pandemic COVID-19 Joins History’s Pandemic Legion,http://doi.org/10.1128/mbio.00812-20,Pandemic C,23.597627
8,eae9dd3b8d09f5e255ab24631fafe10d203752b2,Pre-outbreak determinants of perceived risks of corona infection and preventive measures taken. A prospective population-based study,http://doi.org/10.1371/journal.pone.0234600,Pre-outbre,23.435152
9,edd549ddad8a46749e862be50560fb9aeb2573c1,Global health landscape challenges triggered by COVID-19,http://doi.org/10.1186/s41232-020-00144-5,Global hea,23.256059


,paper_id,title,doi,text,score
0,e26214244bdd92df2c0967ea3e3d0054bfe785f5,In Vitro Diagnostic Assays for COVID-19: Recent Advances and Emerging Trends,http://doi.org/10.3390/diagnostics10040202,In Vitro D,16.425396
1,76e3402f66d6e42dd7459aeb92773811ba094fa4,Effectiveness of COVID-19 diagnosis and management tools: A review,http://doi.org/10.1016/j.radi.2020.09.010,Effectiven,14.450643
2,e2149d1c247ef118df46869e0e8822198f10ad23,The role of peripheral blood eosinophil counts in COVID‐19 patients,http://doi.org/10.1111/all.14465,The role o,14.263009
3,e2930e630933e6fabddd00369308f8250a18a187,Fast Accurate Point of Care COVID-19 Pandemic Diagnosis Enabled Through Advanced Lab-on-a-Chip Optical Biosensors: Opportunities and Challenges,http://doi.org/nan,Fast Accur,14.193659
4,e046a06f262d74d025272ad697d358c4afb5d9bd,Application and optimization of RT-PCR in diagnosis of SARS-CoV-2 infection,http://doi.org/10.1101/2020.02.25.20027755,Applicatio,13.945084
5,e2988719c558bf688974fbd9478a1804439521fc,Navigating the Diagnostics of COVID-19,http://doi.org/10.1007/s42399-020-00408-8,Navigating,13.118135
6,76e34b0b3e937b1062f12fccbaf924503e916bc8,Combined drug repurposing and virtual screening strategies with molecular dynamics simulation identified potent inhibitors for SARS-CoV-2 main protease (3CLpro),http://doi.org/10.1080/07391102.2020.1779128,Combined d,13.046301
7,f0acd81bca3e22cb8a3b480d16437c0f72d46c41,"Lions, Tigers and Kittens too: ACE2 and susceptibility to CoVID-19",http://doi.org/10.1093/emph/eoaa021,"Lions, Tig",12.927674
8,f6ae3e5dde14609bd31e6044688e401c7be3aa2b,COVID-19 IN PATIENTS WITH INFLAMMATORY BOWEL DISEASE: A SYSTEMATIC REVIEW OF CLINICAL DATA,http://doi.org/10.1016/j.dld.2020.09.002,COVID-19 I,12.694453
9,ec04d705e885876d595e4f3df639173ddcb90f03,Facing SARS-CoV-2 outbreak in immunotherapy era,http://doi.org/10.2217/fon-2020-0340,Facing SAR,12.565777


,paper_id,title,doi,text,score
0,e41ef8fb9ea0a8c6abd2159fd94d12a86267806d,Challenges for Microelectronics in Non-Invasive Medical Diagnostics,http://doi.org/10.3390/s20133636,Challenges,29.398444
1,e441a34d18665aa5b139544cdad74ba17d7d4a13,Direct diagnostic testing of SARS-CoV-2 without the need for prior RNA extraction.,http://doi.org/10.1101/2020.05.28.20115220,Direct dia,28.824963
2,e391867b948f595cfe748251586ce599ad3e82ef,"Development and validation of a portable, point-of-care canine distemper virus qPCR test",http://doi.org/10.1371/journal.pone.0232044,Developmen,28.722656
3,f3c018d556cad261ea04fade08d60e5eea7c3859,Point-of-Care Diagnostic Tests for Detecting SARS-CoV-2 Antibodies: A Systematic Review and Meta-Analysis of Real-World Data,http://doi.org/10.3390/jcm9051515,Point-of-C,28.348844
4,e8972b4bfd839e811780fd9ecfc7262152a677ef,Hospital At Home Units In The Post‐Covid 19 ERA,http://doi.org/10.1111/eci.13390,Hospital A,27.705653
5,e775ae09a7e0906313852000f0700011a0415682,A proposal for the return to routine endoscopy during the COVID-19 pandemic,http://doi.org/10.1016/j.gie.2020.04.050,A proposal,27.580749
6,ee024840b40e3086ed1d6feb1134fd7d762edc00,COVID-19 paraclinical diagnostic tools: Updates and future trends,http://doi.org/10.1016/j.retram.2020.06.001,COVID-19 p,27.293134
7,f7cd33f182af02e02408bf4af35b5aad2eddf645,Antibody tests in detecting SARS-CoV-2 infection: a meta-analysis,http://doi.org/10.1101/2020.04.22.20074914,Antibody t,27.002349
8,e33791175d69e3d900ed203acdb00327a1fc96f3,"An easy, reliable and rapid SARS-CoV2 RT-LAMP based test for Point-of-Care and diagnostic lab",http://doi.org/10.1101/2020.09.25.20200956,"An easy, r",26.993584
9,e26214244bdd92df2c0967ea3e3d0054bfe785f5,In Vitro Diagnostic Assays for COVID-19: Recent Advances and Emerging Trends,http://doi.org/10.3390/diagnostics10040202,In Vitro D,26.962000


,paper_id,title,doi,text,score
0,edb4f309c40054108af5620644f06c68c68b0768,Analysis of Spatial-temporal Behavior Pattern of the Share Bike Usage during COVID-19 Pandemic in Beijing,http://doi.org/nan,Analysis o,20.211438
1,f7cbc45f7e4be98a98fc0f9546fa60c25ebbcfda,Isothermal Nucleic Acid Amplification Techniques and Their Use in Bioanalysis,http://doi.org/10.1134/s0006297920020030,Isothermal,17.507062
2,e673817674626d033b5b9c861584bd42da1e5c50,"High Tech, High Risk: Tech Ethics Lessons for the COVID-19 Pandemic Response",http://doi.org/10.1016/j.patter.2020.100102,"High Tech,",15.974420
3,76e34b0b3e937b1062f12fccbaf924503e916bc8,Combined drug repurposing and virtual screening strategies with molecular dynamics simulation identified potent inhibitors for SARS-CoV-2 main protease (3CLpro),http://doi.org/10.1080/07391102.2020.1779128,Combined d,15.477545
4,f0acd81bca3e22cb8a3b480d16437c0f72d46c41,"Lions, Tigers and Kittens too: ACE2 and susceptibility to CoVID-19",http://doi.org/10.1093/emph/eoaa021,"Lions, Tig",14.832875
5,e14f69047a759c01eb0f61c592203181506fb723,In Silico Screening of Some Naturally Occurring Bioactive Compounds Predicts Potential Inhibitors against SARS-COV-2 (COVID-19) Protease,http://doi.org/nan,In Silico,14.828693
6,69f5445e541847993640ec68027ed73ee1980aa5,Trust and Transparency in Contact Tracing Applications,http://doi.org/nan,Trust and,14.738156
7,e2930e630933e6fabddd00369308f8250a18a187,Fast Accurate Point of Care COVID-19 Pandemic Diagnosis Enabled Through Advanced Lab-on-a-Chip Optical Biosensors: Opportunities and Challenges,http://doi.org/nan,Fast Accur,14.723498
8,e83cff607fc35bff49974c0f7283ee1a7e349704,Artificial intelligence predicts the immunogenic landscape of SARS-CoV-2: toward universal blueprints for vaccine designs,http://doi.org/10.1101/2020.04.21.052084,Artificial,14.549002
9,78bcf659c7764369a3044e829791bd4e42e3d559,Contact Tracing Mobile Apps for COVID-19: Privacy Considerations and Related Trade-offs,http://doi.org/nan,Contact Tr,14.410828


,paper_id,title,doi,text,score
0,e84dc78d7df6a96279d846c93f7921763834fe87,"An effective, safe and cost-effective cell-based chimeric vaccine against SARS-CoV2",http://doi.org/10.1101/2020.08.19.258244,An effecti,13.663271
1,e35d6c05896a75cddc4c93d3fb6e7eac55087a95,Detectable severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) in human breast milk of a mildly symptomatic patient with coronavirus disease 2019 (COVID-19),http://doi.org/10.1093/cid/ciaa673,Detectable,13.238074
2,eae0effca94e696cb437cf256e70428b3dd02e29,Mapping the Global Research and Clinical Trials in COVID-19,http://doi.org/10.1101/2020.06.27.20141788,Mapping th,13.235497
3,e83cff607fc35bff49974c0f7283ee1a7e349704,Artificial intelligence predicts the immunogenic landscape of SARS-CoV-2: toward universal blueprints for vaccine designs,http://doi.org/10.1101/2020.04.21.052084,Artificial,13.223823
4,e421194b230d5a0a59ea097587a76da65e3e0b3b,Clinical characteristics and humoral immune response in Healthcare Workers with COVID-19 in a Teaching Hospital in Belgium,http://doi.org/10.1016/j.jhin.2020.09.018,Clinical c,12.868182
5,e69e83feb23c448fd846dcb17a577ea3024442ab,Profile of specific antibodies to SARS-CoV-2: The first report,http://doi.org/10.1016/j.jinf.2020.03.012,Profile of,12.816554
6,f2ca707d0b5e2eb201ca53ca919b763b2fd8ad9e,"Cerebellar Infarction Requiring Surgical Decompression in patient with COVID 19 Pathological Analysis, Brief Review",http://doi.org/10.1016/j.inat.2020.100850,Cerebellar,12.802594
7,e90a11bbc7b9cec4e674a515fbe317549f022be8,First results from the UK COVID-19 Serology in Oncology Staff Study (CSOS),http://doi.org/10.1101/2020.06.22.20136838,First resu,12.800850
8,ec04d705e885876d595e4f3df639173ddcb90f03,Facing SARS-CoV-2 outbreak in immunotherapy era,http://doi.org/10.2217/fon-2020-0340,Facing SAR,12.458333
9,81ac9d5d89284a337ac638bfb5becf3e12d330de,Coinfection of SARS-CoV-2 and Other Respiratory Pathogens,http://doi.org/10.2147/idr.s267238,Coinfectio,12.405272


,paper_id,title,doi,text,score
0,ebadaaaf1d5948fae5ecf78a61d18a7db3e0ca4b,A Minimalist Strategy Towards Temporarily Defining Protection for COVID-19,http://doi.org/10.1007/s42399-020-00533-4,A Minimali,42.445346
1,71a10e9373bea71c5a30cc9de81c794764d63ebc,Practical Aspects and Considerations When Planning a New Clinical Microbiology Laboratory,http://doi.org/10.1016/j.cll.2020.08.015,Practical,40.955022
2,77ad3841ab962b29f5ef7af6c72e69845b7f1805,Molecular Testing for Acute Respiratory Tract Infections: Clinical and Diagnostic Recommendations From the IDSA’s Diagnostics Committee,http://doi.org/10.1093/cid/ciaa508,Molecular,40.066180
3,77b1e88254e4aa07d5841d2c9a09de09bf6069d3,"Associations between psychiatric disorders, COVID-19 testing probability and COVID-19 testing results: findings from a population-based study",http://doi.org/10.1192/bjo.2020.75,Associatio,39.316024
4,70b66d624b96569304621663178b5759e3dd1d3f,The Epistemology of a Positive SARS-CoV-2 Test,http://doi.org/10.1007/s10441-020-09393-w,The Episte,39.036207
5,ead4eb0d9efefd60ae894fc2f3e685f1d47128bb,COVID-19 AND THE GLOBAL IMPACT ON COLORECTAL PRACTICE AND SURGERY,http://doi.org/10.1016/j.clcc.2020.05.011,COVID-19 A,38.394965
6,e42c5bb6d800efa745214caa8fc413c87f8984bd,"Leishmaniasis diagnosis: an update on the use of parasitological, immunological and molecular methods",http://doi.org/10.1007/s12639-020-01212-w,Leishmania,38.370999
7,f8e259c6559c2d79f12c98483bd2e340639cf549,Testing for SARS-CoV-2 (COVID-19): a systematic review and clinical guide to molecular and serological in-vitro diagnostic assays,http://doi.org/10.1016/j.rbmo.2020.06.001,Testing fo,38.115811
8,80c0361651ecf3fed3e4e08b6f6cb358389037b2,The Status Dynamics of Role Blurring in the Time of COVID-19,http://doi.org/10.1177/2378023120944358,The Status,38.053427
9,e528052f7dfa598b11d4a0402607cbc514877724,Evaluation of machine learning algorithms for Health and Wellness applications: a tutorial,http://doi.org/nan,Evaluation,37.975280


# BERT For Question Answering

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 7.2MB/s 
     |████████████████████████████████| 890kB 35.6MB/s 
     |████████████████████████████████| 2.9MB 31.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=fac71ec5acf801ddd9adb6d9c56ccee6ed83c8460e96d892786578969ccbcf66
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [ ]:
from transformers import BertForQuestionAnswering

model2 = BertForQuestionAnswering.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [ ]:
from transformers import BertForQuestionAnswering

model3 = BertForQuestionAnswering.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
from transformers import BertForQuestionAnswering

model4 = BertForQuestionAnswering.from_pretrained('bert-large-cased')

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-cased and

In [ ]:
from transformers import BertForQuestionAnswering

model5= BertForQuestionAnswering.from_pretrained('bert-base-cased-finetuned-mrpc')

Some weights of the model checkpoint at bert-base-cased-finetuned-mrpc were not used when initializing BertForQuestionAnswering: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased-finetuned-mrpc and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import BertTokenizer

tokenizer3 = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
from transformers import BertTokenizer

tokenizer4 = BertTokenizer.from_pretrained('bert-large-cased')

In [ ]:
from transformers import BertTokenizer

tokenizer5 = BertTokenizer.from_pretrained('bert-base-cased-finetuned-mrpc')

In [ ]:
def answer_question(question,text):
    input_ids = tokenizer.encode(question,text)

    #print(input_ids)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    tokens=tokens[:511]
    #print('The input has a total of {:} tokens.'.format(len(input_ids)))

    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    segment_ids=segment_ids[:511]
    input_ids=input_ids[:511]


    #print(len(segment_ids))
    #print(len(input_ids))
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # Run our example through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids]),return_dict=False) # The segment IDs to differentiate question from answer_text

    # Find the tokens with the highest `start` and `end` scores.
    #print(start_scores)
    #print(end_scores)
    answer_start = torch.argmax(start_scores)
    

    #print(answer_start)
    answer_end = torch.argmax(end_scores)
    
    #print(answer_end)

    # Combine the tokens in the answer and print it out.
    answer = ' '.join(tokens[answer_start:answer_end+1])

    #print(len(answer))
    #if(len(answer))
    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    #print('Answer: "' + answer + '"') 
    #print (answer)
    if(len(answer)<50):
      answer="Appropriate answer not found"
    return answer

In [ ]:
def answer_question2(question,text):
    #question = "[CLS] " + question
    #answer_text=" [SEP] " + text + " [SEP]"
    input_ids = tokenizer.encode(question,text)

    #print(input_ids)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    tokens=tokens[:511]
    #print('The input has a total of {:} tokens.'.format(len(input_ids)))

    

    # BERT only needs the token IDs, but for the purpose of inspecting the 
    # tokenizer's behavior, let's also get the token strings and display them.

    # For each token and its id...
    #for token, id in zip(tokens, input_ids):
        
        # If this is the [SEP] token, add some space around it to make it stand out.
        #if id == tokenizer.sep_token_id:
            #print('')
        
        # Print the token string and its ID in two columns.
        #print('{:<12} {:>6,}'.format(token, id))

        #if id == tokenizer.sep_token_id:
            #print('')



    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    segment_ids=segment_ids[:511]
    input_ids=input_ids[:511]


    #print(len(segment_ids))
    #print(len(input_ids))
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # Run our example through the model.
    start_scores, end_scores = model2(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids]),return_dict=False) # The segment IDs to differentiate question from answer_text

    # Find the tokens with the highest `start` and `end` scores.
    #print(start_scores)
    #print(end_scores)
    answer_start = torch.argmax(start_scores)
    

    #print(answer_start)
    answer_end = torch.argmax(end_scores)
    
    #print(answer_end)

    # Combine the tokens in the answer and print it out.
    answer = ' '.join(tokens[answer_start:answer_end+1])

    #print(len(answer))
    #if(len(answer))
    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    #print('Answer: "' + answer + '"') 
    #print (answer)
    if(len(answer)<10):
      answer="Appropriate answer not found"
    return answer

In [ ]:
def answer_question3(question,text):
    #question = "[CLS] " + question
    #answer_text=" [SEP] " + text + " [SEP]"
    input_ids = tokenizer3.encode(question,text)

    #print(input_ids)
    tokens = tokenizer3.convert_ids_to_tokens(input_ids)
    tokens=tokens[:511]
    #print('The input has a total of {:} tokens.'.format(len(input_ids)))

    

    # BERT only needs the token IDs, but for the purpose of inspecting the 
    # tokenizer's behavior, let's also get the token strings and display them.

    # For each token and its id...
    #for token, id in zip(tokens, input_ids):
        
        # If this is the [SEP] token, add some space around it to make it stand out.
        #if id == tokenizer.sep_token_id:
            #print('')
        
        # Print the token string and its ID in two columns.
        #print('{:<12} {:>6,}'.format(token, id))

        #if id == tokenizer.sep_token_id:
            #print('')



    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer3.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    segment_ids=segment_ids[:511]
    input_ids=input_ids[:511]


    #print(len(segment_ids))
    #print(len(input_ids))
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # Run our example through the model.
    start_scores, end_scores = model3(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids]),return_dict=False) # The segment IDs to differentiate question from answer_text

    # Find the tokens with the highest `start` and `end` scores.
    #print(start_scores)
    #print(end_scores)
    answer_start = torch.argmax(start_scores)
    

    #print(answer_start)
    answer_end = torch.argmax(end_scores)
    
    #print(answer_end)

    # Combine the tokens in the answer and print it out.
    answer = ' '.join(tokens[answer_start:answer_end+1])

    #print(len(answer))
    #if(len(answer))
    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    #print('Answer: "' + answer + '"') 
    #print (answer)
    if(len(answer)<50):
      answer="Appropriate answer not found"
    return answer

In [ ]:
def answer_question4(question,text):
    #question = "[CLS] " + question
    #answer_text=" [SEP] " + text + " [SEP]"
    input_ids = tokenizer4.encode(question,text)

    #print(input_ids)
    tokens = tokenizer4.convert_ids_to_tokens(input_ids)
    tokens=tokens[:511]
    #print('The input has a total of {:} tokens.'.format(len(input_ids)))

    

    # BERT only needs the token IDs, but for the purpose of inspecting the 
    # tokenizer's behavior, let's also get the token strings and display them.

    # For each token and its id...
    #for token, id in zip(tokens, input_ids):
        
        # If this is the [SEP] token, add some space around it to make it stand out.
        #if id == tokenizer.sep_token_id:
            #print('')
        
        # Print the token string and its ID in two columns.
        #print('{:<12} {:>6,}'.format(token, id))

        #if id == tokenizer.sep_token_id:
            #print('')



    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer4.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    segment_ids=segment_ids[:511]
    input_ids=input_ids[:511]


    #print(len(segment_ids))
    #print(len(input_ids))
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # Run our example through the model.
    start_scores, end_scores = model4(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids]),return_dict=False) # The segment IDs to differentiate question from answer_text

    # Find the tokens with the highest `start` and `end` scores.
    #print(start_scores)
    #print(end_scores)
    answer_start = torch.argmax(start_scores)
    

    #print(answer_start)
    answer_end = torch.argmax(end_scores)
    
    #print(answer_end)

    # Combine the tokens in the answer and print it out.
    answer = ' '.join(tokens[answer_start:answer_end+1])

    #print(len(answer))
    #if(len(answer))
    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    #print('Answer: "' + answer + '"') 
    #print (answer)
    if(len(answer)<50):
      answer="Appropriate answer not found"
    return answer

In [ ]:
def answer_question5(question,text):
    #question = "[CLS] " + question
    #answer_text=" [SEP] " + text + " [SEP]"
    input_ids = tokenizer5.encode(question,text)

    #print(input_ids)
    tokens = tokenizer5.convert_ids_to_tokens(input_ids)
    tokens=tokens[:511]
    #print('The input has a total of {:} tokens.'.format(len(input_ids)))

    

    # BERT only needs the token IDs, but for the purpose of inspecting the 
    # tokenizer's behavior, let's also get the token strings and display them.

    # For each token and its id...
    #for token, id in zip(tokens, input_ids):
        
        # If this is the [SEP] token, add some space around it to make it stand out.
        #if id == tokenizer.sep_token_id:
            #print('')
        
        # Print the token string and its ID in two columns.
        #print('{:<12} {:>6,}'.format(token, id))

        #if id == tokenizer.sep_token_id:
            #print('')



    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer5.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    segment_ids=segment_ids[:511]
    input_ids=input_ids[:511]


    #print(len(segment_ids))
    #print(len(input_ids))
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # Run our example through the model.
    start_scores, end_scores = model5(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids]),return_dict=False) # The segment IDs to differentiate question from answer_text

    # Find the tokens with the highest `start` and `end` scores.
    #print(start_scores)
    #print(end_scores)
    answer_start = torch.argmax(start_scores)
    

    #print(answer_start)
    answer_end = torch.argmax(end_scores)
    
    #print(answer_end)

    # Combine the tokens in the answer and print it out.
    answer = ' '.join(tokens[answer_start:answer_end+1])

    #print(len(answer))
    #if(len(answer))
    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    #print('Answer: "' + answer + '"') 
    #print (answer)
    if(len(answer)<10):
      answer="Appropriate answer not found"
    return answer

# Result obtained from BERT Large Cased

In [ ]:
from IPython.core.display import display, HTML

pd.set_option('display.max_colwidth',None)
m=0
for i in range(0,len(doc_list)):
  finaldf=pd.DataFrame(columns=['paper_id','title','doi','text','score','answer_exercpt'])
  display(HTML('<h1> '+queries[i]+'</h1>'))
  k=0
  for id in doc_list[i]:
    for index,row in df_dict[id].T.iterrows():
      link=str(row['doi'])
      title=row["title"]
      linka='http://doi.org/'+link
      final_link=linka
      answer=answer_question4(queries[m],row['text'])
      to_append = [row['paper_id'],title,final_link,row['text'][:10],top10_scores_sorted[i][k],answer]
      df_length = len(finaldf)
      finaldf.loc[df_length] = to_append
      k+=1
  m+=1
  display(finaldf)


Token indices sequence length is longer than the specified maximum sequence length for this model (1844 > 512). Running this sequence through the model will result in indexing errors


,paper_id,title,doi,text,score,answer_exercpt
0,e86a2d96cb6bea7787c58496b384c59b788017dd,COVID-19: Global Health Equity in Pandemic Response,http://doi.org/10.4269/ajtmh.20-0260,COVID-19:,27.669937,"why tuberculosis still remains a major public health concern despite the advent of effective therapy for drug - sensitive disease almost 50 years ago . 3 As the world races to find an effective antiviral against severe acute respiratory syndrome coronavirus - 2 , as well as a vaccine , will these become sovereign commodities of the global North ? We see no reason why injustices of the past will not be repeated just because the pathogen is novel . Often , widespread access to lifesaving therapeutics - like treatment for HIV infection - has only been made possible for marginalized groups through hard - fought activism . But activism alone is not enough - in 2019 , the U . S . government declined influenza vaccinations to de"
1,e54cfdf6d59ae0174571e71ead0061a9b32b80c3,PPE or not PPE - that is the question,http://doi.org/10.1038/s41415-020-1639-y,PPE or not,25.250862,Appropriate answer not found
2,eb819f89014a6e662d0208c962155b91e91d6584,Singing Away the Social Distancing Blues: Art Therapy in a Time of Coronavirus,http://doi.org/10.1177/0022167820927807,Singing Aw,24.547355,"##siliency , and hope , though the crisis is still in its early stages . In response to the helplessness , uncertainty , and fear caused by the new coronavirus , health care professionals are be"
3,ec59134c9675007a1de94a4fcce28c139fb1baeb,"The Mortal Coil of Covid-19, Fake News, and Negative Epistemic Postdigital Inculcation",http://doi.org/10.1007/s42438-020-00192-7,The Mortal,24.442839,Appropriate answer not found
4,f8c120006ddc1c256306a35759e00aaa505b9c6b,Coronavirus Pandemic as Black Swan Event,http://doi.org/10.1007/978-3-030-49264-9_32,Coronaviru,23.811713,"On another side some people said that coronavirus is no scared than simple influence . Where is true ? The health effects and mortality of coronavirus and influenza were compared . Situation Reports of World Health Organization have been analyzed and signs of Black Swan event considered . Total confirmed cases , total , deaths , and Rate of coronavirus distribution was calculated based on Situation Reports of the World Health Organization . To conclude this paper Coronavirus Pandemic recognized as Black Swan event according to considered figures and facts from recent references . IntroductionThere is a worldwide concern about the new coronavirus 2019 - nCoV as a global public health threat . The 2019 - nCoV infection is spreading and its incidence is increasing worldwide . Although the current mortality is lower than that of the SARS - CoV and the MERS - CoV , it seems that the 2019 - nCoV is very contagious . SARSCoV ( severe acute respiratory syndrome coronavirus ) was recognized in November 2002 , MERS - CoV ( Middle East respiratory syndrome coronavirus ) in June 2012 , and 2019 - nCoV in December 2019 . The first deaths occurred mostly in elderly people , among whom the disease might progress faster . But now it established that the young people are also prone to severe coronavirus disease so the society should still be cautious in dealing with the virus and pay more attention to protecting all of people from the virus . IntroductionCoronavirus spread worldwide with the pandemic according to data of World Health Organization ( WHO ) . Since Coronavirus is spreading very fast , 11 . 03 . 2020 WHO Director - General ' s characterized Coronavirus as a pandemic and this pandemic has the social and economic impacts [ 5 , report 51 , 71 ] . IntroductionThere are signs"
5,f4c762289a704d61cde8da3ddae21fc7a0ed5e99,Psychologist Self-Care During the Pandemic: Now More Than Ever,http://doi.org/10.1007/s42843-020-00010-5,Psychologi,23.799986,"ailing parents , and by her own anxiety , which is heightened by the knowledge that the African - American community has been differentially affected by Samuel Jones , PhD ,

,paper_id,title,doi,text,score,answer_exercpt
0,e26214244bdd92df2c0967ea3e3d0054bfe785f5,In Vitro Diagnostic Assays for COVID-19: Recent Advances and Emerging Trends,http://doi.org/10.3390/diagnostics10040202,In Vitro D,16.425396,Appropriate answer not found
1,76e3402f66d6e42dd7459aeb92773811ba094fa4,Effectiveness of COVID-19 diagnosis and management tools: A review,http://doi.org/10.1016/j.radi.2020.09.010,Effectiven,14.450643,"[CLS] New advances in diagnosing SARS - COV - 2 [SEP] Effectiveness of COVID - 19 diagnosis and management tools : A review Objective : to review the available literature concerning the effectiveness of the COVID - 19 diagnostic tools Background With the absence of specific treatment / vaccines for the coronavirus COVID - 19 , the most appropriate approach to control this infection is to quarantine people and is"
2,e2149d1c247ef118df46869e0e8822198f10ad23,The role of peripheral blood eosinophil counts in COVID‐19 patients,http://doi.org/10.1111/all.14465,The role o,14.263009,Appropriate answer not found
3,e2930e630933e6fabddd00369308f8250a18a187,Fast Accurate Point of Care COVID-19 Pandemic Diagnosis Enabled Through Advanced Lab-on-a-Chip Optical Biosensors: Opportunities and Challenges,http://doi.org/nan,Fast Accur,14.193659,[CLS] New advances in diagnosing SARS - COV - 2 [SEP] Fast Accurate Point of Care COVID - 19 Pandemic Diagnosis Enabled Through Advanced Lab - on - a - Chip Optical Biosensors : Opportunities and Challenges The sudden rise of severe acute respiratory syndrome coronavirus 2 ( SARS - CoV - 2 ) pandemic early 2020 throughout the world has called into drastic action measures to do instant detection and reduce the spread rate . The common diagnostics testing methods has been only partially effective in satisfying the booming demand for fast detection methods
4,e046a06f262d74d025272ad697d358c4afb5d9bd,Application and optimization of RT-PCR in diagnosis of SARS-CoV-2 infection,http://doi.org/10.1101/2020.02.25.20027755,Applicatio,13.945084,"79 / 86 ) than RT - PCR alone ( 78 . 2 % ， 68 / 87 ) or CT alone ( 66 . 7 % , 54 of 81 ) or combination of two RT - PCR tests ( 86 . 2 % , 75 / 87 ) . There was good agreement between RT - PCR and CT ( kappa - value , 0 . 430 ) . In 34 COVID - 19 cases with inconsistent results , 94 . 1 % ( n = 32 ) are mild infection , 62 . 5 % of which ( 20 / 32 ) showed positive RT - PCR . 46 . 7 % ( 35 / 75 ) COVID - 19 patients had at least one positive stool during the course . Two cases had positive stool earlier than the pharyngeal s"
5,e2988719c558bf688974fbd9478a1804439521fc,Navigating the Diagnostics of COVID-19,http://doi.org/10.1007/s42399-020-00408-8,Navigating,13.118135,"individuals infected and exceeding 400 , 000 deaths globally , the necessity to develop fast and efficient diagnostic methods is of high importance . This paper reports"
6,76e34b0b3e937b1062f12fccbaf924503e916bc8,Combined drug repurposing and virtual screening strategies with molecular dynamics simulation identified potent inhibitors for SARS-CoV-2 main protease (3CLpro),http://doi.org/10.1080/07391102.2020.1779128,Combined d,13.046301,"##avir , and five drugs ( TCM5280805 , TCM5280445 , TCM5280343 , TCM5280863 , and TCM5458190 ) from the TCM database were found"
7,f0acd81bca3e22cb8a3b480d16437c0f72d46c41,"Lions, Tigers and Kittens too: ACE2 and susceptibility to CoVID-19",http://doi.org/10.1093/emph/eoaa021,"Lions, Tig",12.927674,"##E2 and susceptibility to CoVID - 19 SARS - CoV - 2 ( Severe Acute Respiratory Syndrome coronavirus 2 ) has been reported to infect domesticated animals in a species - specific manner , where cats were susceptible but not dogs . Using the recently published crystal structure of the SARS - CoV - 2 spike protein complexed with the human host cell receptor ACE2 , we characterized the structure and evolution of ACE2 in several of these species and identify a single interacting amino acid residue conserved between human and Felidae ACE2 but not in Canidae that co

,paper_id,title,doi,text,score,answer_exercpt
0,e41ef8fb9ea0a8c6abd2159fd94d12a86267806d,Challenges for Microelectronics in Non-Invasive Medical Diagnostics,http://doi.org/10.3390/s20133636,Challenges,29.398444,"sometimes with changing fortunes , as a key enabling technology in diagnostics . This paper reviews"
1,e441a34d18665aa5b139544cdad74ba17d7d4a13,Direct diagnostic testing of SARS-CoV-2 without the need for prior RNA extraction.,http://doi.org/10.1101/2020.05.28.20115220,Direct dia,28.824963,Appropriate answer not found
2,e391867b948f595cfe748251586ce599ad3e82ef,"Development and validation of a portable, point-of-care canine distemper virus qPCR test",http://doi.org/10.1371/journal.pone.0232044,Developmen,28.722656,Appropriate answer not found
3,f3c018d556cad261ea04fade08d60e5eea7c3859,Point-of-Care Diagnostic Tests for Detecting SARS-CoV-2 Antibodies: A Systematic Review and Meta-Analysis of Real-World Data,http://doi.org/10.3390/jcm9051515,Point-of-C,28.348844,"54 . 5 – 74 . 0 ) , and specificity of 98 . 0 % ( 95 % CI 95 . 8 – 99 . 0 ) , with high heterogeneity and risk of reporting bias . We can conclude that : rapid diagnostic tests for COVID - 19 are necessary , but should be adequately sensitive and specific ; few studies have been carried out to date ; the studies included are characterized by low numbers and low sample power , and in light of these results , the use of available tests is currently questionable for clinical purposes and cannot substitute other more reliable molecular tests , such as assays based on RT - PCR . Abstract : SARS - CoV - 2 is responsible for a highly con"
4,e8972b4bfd839e811780fd9ecfc7262152a677ef,Hospital At Home Units In The Post‐Covid 19 ERA,http://doi.org/10.1111/eci.13390,Hospital A,27.705653,"date . The World Health Organization declared COVID ‐ 19 as a Public Health Emergency of International Concern on January 30 ( th ) , 2020 and on March 11 ( th ) COVID ‐ 19 was declared a pandemic disease . In Spain , first cases were diagnosed in early February , with the incidence peak occurring between the last week of March and the first week of April . At the time of writing this article 227 , 000 patients were infected and 26 , 000 patients had died of COVID ‐ 19 . The rapid spread of the virus and the severity of the disease has been a real challenge to the health systems of most affected countries , compelling to the transformation of hospitals in COVID ‐ 19 monographic centres . In order to cover COVID ‐ 19 overwhelming necessities , non ‐ COVID activity was reduced to the minimum , cancelling non ‐ urgent surgery , outpatient clinics and treatments , and diagnostic tests . Since the start of the SARS - CoV - 2 outbreak in China in December 2019 COVID - 19 has affected nearly 4 , 5 million people around the world , with almost 286 , 000 fatalities reported as of the current date . The World Health Organization declared COVID - 19 as a Public Health Emergency of International Concern on January 30 th , 2020 and on March 11 th COVID - 19 was declared a pandemic disease . In Spain , first cases were diagnosed in early February , with the incidence peak occurring between the last week of March and the first week of April . At the time of writing this article 227 , 000 patients were infected and 26 , 000 patients had died of COVID - 19 . The rapid spread of the virus and the severity of the disease has been a real challenge to the health systems of most affected countries , compelling to the transformation of hospitals in COVID - 19 mon"
5,e775ae09a7e0906313852000f0700011a0415682,A proposal for the return to routine endoscopy during the COVID-19 pandemic,http://doi.org/10.1016/j.gie.2020.04.050,A proposal,27.580749,Appropriate answer not found
6,ee024840b40e3086ed1d6feb1134fd7d762edc00,COVID-19 paraclinical diagnostic tools: Updates and future trends,http://doi.org/10.1016/j.retram.2020.06.001,COVID-19 p,27.293134,Appropriate answer not found
7,f7cd33f182af02e02408bf4af35b5aad2eddf645,An

,paper_id,title,doi,text,score,answer_exercpt
0,edb4f309c40054108af5620644f06c68c68b0768,Analysis of Spatial-temporal Behavior Pattern of the Share Bike Usage during COVID-19 Pandemic in Beijing,http://doi.org/nan,Analysis o,20.211438,Appropriate answer not found
1,f7cbc45f7e4be98a98fc0f9546fa60c25ebbcfda,Isothermal Nucleic Acid Amplification Techniques and Their Use in Bioanalysis,http://doi.org/10.1134/s0006297920020030,Isothermal,17.507062,Appropriate answer not found
2,e673817674626d033b5b9c861584bd42da1e5c50,"High Tech, High Risk: Tech Ethics Lessons for the COVID-19 Pandemic Response",http://doi.org/10.1016/j.patter.2020.100102,"High Tech,",15.974420,"8 The risks that define contemporary life are not just socially produced , but are actively managed by"
3,76e34b0b3e937b1062f12fccbaf924503e916bc8,Combined drug repurposing and virtual screening strategies with molecular dynamics simulation identified potent inhibitors for SARS-CoV-2 main protease (3CLpro),http://doi.org/10.1080/07391102.2020.1779128,Combined d,15.477545,"##avir , and five drugs ( TCM5280805 , TCM5280445 , TCM5280343 , TCM5280863 , and TCM5458190 ) from the TCM database were found"
4,f0acd81bca3e22cb8a3b480d16437c0f72d46c41,"Lions, Tigers and Kittens too: ACE2 and susceptibility to CoVID-19",http://doi.org/10.1093/emph/eoaa021,"Lions, Tig",14.832875,"##E2 and susceptibility to CoVID - 19 SARS - CoV - 2 ( Severe Acute Respiratory Syndrome coronavirus 2 ) has been reported to infect domesticated animals in a species - specific manner , where cats were susceptible but not dogs . Using the recently published crystal structure of the SARS - CoV - 2 spike protein complexed with the human host cell receptor ACE2 , we characterized the structure and evolution of ACE2 in several of these species and identify a single interacting amino acid residue conserved between human and Felidae ACE2 but not in Canidae that correlates with virus susceptibility . Using computational analyses we describe how this site likely affects ACE2 targeting by the virus . Thus , we highlight how evolution - based approaches can be used to form hypotheses and study animal transmission of such viruses in the future . Lay summary : Recently , the virus that causes COVID - 19 was shown to transmit to companion animals . A single genetic change in the host receptor for the virus inherited in cats , but not dogs , correlates with feline susceptibility . With human - to - cat transmission of COVID - 19 confirmed , such information can inform public health policy regarding companion animals . Abstract : SARS - CoV - 2 ( Severe Acute Respiratory Syndrome coronavirus 2 ) has been reported to infect domesticated animals in a species - specific manner , where cats were susceptible but not dogs . Using the recently published crystal structure of the SARS - CoV - 2 spike protein complexed with the human host cell receptor ACE2 , we characterized the structure and evolution of ACE2 in several of these species and identify a single interacting amino acid residue conserved between human and Felidae ACE2 but not in Canidae that correlates with virus susceptibility . Using computational analyses we describe how this site likely affects ACE2 targeting by the virus . Thus , we highlight how evolution - based approaches can be used to form hypotheses and study animal transmission of such viruses in the future . Brevia : A study led by Bu Zhigao and colleagues examined the ability of SARS - CoV - 2 , which is"
5,e14f69047a759c01eb0f61c592203181506fb723,In Silico Screening of Some Naturally Occurring Bioactive Compounds Predicts Potential Inhibitors against SARS-COV-2 (COVID-19) Protease,http://doi.org/nan,In Silico,14.828693,"631 + G ( d , p ) level we have performed their molecular docking with SARS - COV - 2 protease to calculate the binding affinity as well as to screen the binding at S - protein site during ligand - protein interactions . Out of these nine studied naturally occurring compounds ; Oleanic Acid 

,paper_id,title,doi,text,score,answer_exercpt
0,e84dc78d7df6a96279d846c93f7921763834fe87,"An effective, safe and cost-effective cell-based chimeric vaccine against SARS-CoV2",http://doi.org/10.1101/2020.08.19.258244,An effecti,13.663271,Appropriate answer not found
1,e35d6c05896a75cddc4c93d3fb6e7eac55087a95,Detectable severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) in human breast milk of a mildly symptomatic patient with coronavirus disease 2019 (COVID-19),http://doi.org/10.1093/cid/ciaa673,Detectable,13.238074,"year old female with no significant medical problems returned to Australia with her eight - month old son in early March 2020 from a COVID - 19 endemic region after a two - month stay . Three days following her return , she developed"
2,eae0effca94e696cb437cf256e70428b3dd02e29,Mapping the Global Research and Clinical Trials in COVID-19,http://doi.org/10.1101/2020.06.27.20141788,Mapping th,13.235497,papers and ongoing clinical trials to provide an informed view of the current status of the research and drug development activity as seen through the lens of research papers and clinical trials . The intended goal of this study is to help the research community and policy makers to keep track of highly relevant research and drug development in COVID - 19 . IntroductionThe intended goal of this study is to help the research community and policy makers to keep track
3,e83cff607fc35bff49974c0f7283ee1a7e349704,Artificial intelligence predicts the immunogenic landscape of SARS-CoV-2: toward universal blueprints for vaccine designs,http://doi.org/10.1101/2020.04.21.052084,Artificial,13.223823,Appropriate answer not found
4,e421194b230d5a0a59ea097587a76da65e3e0b3b,Clinical characteristics and humoral immune response in Healthcare Workers with COVID-19 in a Teaching Hospital in Belgium,http://doi.org/10.1016/j.jhin.2020.09.018,Clinical c,12.868182,"antibodies [SEP] Clinical characteristics and humoral immune response in Healthcare Workers with COVID - 19 in a Teaching Hospital in Belgium BACKGROUND : Healthcare workers ( HCWs ) are at high risk of acquiring COVID - 19 and could play a role in nosocomial transmission . Since February 4 , 2020 , Belgian Health authorities reported more than 90568 cases , of which 8 . 3 % were HCWs . Data on clinical characteristics , sources of infection and humoral immune response of HCWs with COVID - 19 remain scarce . AIM : Analyse the clinical characteristics , humoral immune response , sources of contamination , and outcomes among HCWs with COVID - 19 . METHODS : This retrospective study includes 176 HCWs with laboratory - confirmed COVID - 19 in a teaching hospital in Belgium . Between March 1 ( st ) , and May 31 ( th ) 2020 , all HCWs with symptoms suspect of COVID - 19 were tested through a RT - PCR on a nasopharyngeal swab . Serological testing was performed between 55 and 137 days after the onset of symptoms . FINDINGS :"
5,e69e83feb23c448fd846dcb17a577ea3024442ab,Profile of specific antibodies to SARS-CoV-2: The first report,http://doi.org/10.1016/j.jinf.2020.03.012,Profile of,12.816554,"date after onset of symptoms to detect the specific antibodies to SARS - CoV - 2 . IgM and IgG were analyzed by chemiluminescent immunoassay according to the manufacturer ' s protocol ( Shenzhen Yahuilong Biotechnology Co . , Ltd ) . All data ( test dates and results of IgM and IgG ) were collected up to the final follow - up date ( March 3rd , 2020 ) . Details of demographic characteristics and test dates and results of IgM and IgG were list"
6,f2ca707d0b5e2eb201ca53ca919b763b2fd8ad9e,"Cerebellar Infarction Requiring Surgical Decompression in patient with COVID 19 Pathological Analysis, Brief Review",http://doi.org/10.1016/j.inat.2020.100850,Cerebellar,12.802594,"[CLS] Diagnosing SARS - COV - 2 with antibodies [SEP] Cerebellar Infarction Requiring Surgical Decompression in patient with COVID 19 Pathological Analysis , Brief Review BACKGROUND : This report and literature review describes a ca

,paper_id,title,doi,text,score,answer_exercpt
0,ebadaaaf1d5948fae5ecf78a61d18a7db3e0ca4b,A Minimalist Strategy Towards Temporarily Defining Protection for COVID-19,http://doi.org/10.1007/s42399-020-00533-4,A Minimali,42.445346,"individuals can allow for the definition of protective thresholds of neutralizing antibody . Thereafter , many other antibody assays will be able to screen for surrogate protection after correlations with protective neutralizing antibody are made . Specificity of common antibody tests would benefit from confirmatory blocking systems or confirmatory immunoblotting fingerprints with well - defined antigen ( s"
1,71a10e9373bea71c5a30cc9de81c794764d63ebc,Practical Aspects and Considerations When Planning a New Clinical Microbiology Laboratory,http://doi.org/10.1016/j.cll.2020.08.015,Practical,40.955022,"? While you can participate in discussions aimed at providing answers to such questions , the combined vision and input of hospital and departmental leadership is mandatory . Likewise , if the centralized laboratory is "" off - site "" from 1 or more hospitals , it is necessary to have agreement on what testing capabilities will remain at the individual hospitals ; distance from hospitals to the new central laboratory and frequency and dependability of courier service will be key factors in determining what testing remains at individual hospitals in the system . Without clarity on these issues , the end product , that is , the new laboratory , will likely not meet"
2,77ad3841ab962b29f5ef7af6c72e69845b7f1805,Molecular Testing for Acute Respiratory Tract Infections: Clinical and Diagnostic Recommendations From the IDSA’s Diagnostics Committee,http://doi.org/10.1093/cid/ciaa508,Molecular,40.066180,Appropriate answer not found
3,77b1e88254e4aa07d5841d2c9a09de09bf6069d3,"Associations between psychiatric disorders, COVID-19 testing probability and COVID-19 testing results: findings from a population-based study",http://doi.org/10.1192/bjo.2020.75,Associatio,39.316024,Appropriate answer not found
4,70b66d624b96569304621663178b5759e3dd1d3f,The Epistemology of a Positive SARS-CoV-2 Test,http://doi.org/10.1007/s10441-020-09393-w,The Episte,39.036207,"8 ) for V against ¬V regardless of sensitivity . Accordingly , low specificity tests could not provide strong evidence in favor of C in all plausible scenarios modeled . We also show"
5,ead4eb0d9efefd60ae894fc2f3e685f1d47128bb,COVID-19 AND THE GLOBAL IMPACT ON COLORECTAL PRACTICE AND SURGERY,http://doi.org/10.1016/j.clcc.2020.05.011,COVID-19 A,38.394965,"April 16 - 28 , 2020 . RESULTS : 287 surgeons completed the survey . 90 % were colorectal specialists or general surgeons with an interest in colorectal diseases . COVID - 19 had impacted the practice of 96 % surgeons and 52 % were now using telemedicine . 66 % stated that elective colorectal cancer surgery ( ECCS ) could proceed but with perioperative precautions . 19 . 5 % of respondents stated that provision of personal protection equipment was the most important perioperative precaution , however , this was only being provided in 9 . 1 % of hospitals . 64 % of surgeons were offering minimally invasive surgery , however , 44 % stated that there was not enough information about the safety of loss of intrabdominal CO2 gas during the COVID - 19 pandemic . 61 % of surgeons were prepared to defer ECCS , with 29 % willing to defer for up to 8 weeks . CONCLUSIONS : Our survey has demonstrated that globally COVID - 19 has affected the ability of colorec"
6,e42c5bb6d800efa745214caa8fc413c87f8984bd,"Leishmaniasis diagnosis: an update on the use of parasitological, immunological and molecular methods",http://doi.org/10.1007/s12639-020-01212-w,Leishmania,38.370999,Appropriate answer not found
7,f8e259c6559c2d79f12c98483bd2e340639cf549,Testing for SARS-CoV-2 (COVID-19): a systematic review and clinical guide to molecular and serological in-vitro diagnostic assays,http://doi.org/10.1016/j.rbmo.2020.06.001,Testing fo,38.115811,"002 ,

# Result obtained from BERT Base UnCased

In [ ]:
from IPython.core.display import display, HTML

pd.set_option('display.max_colwidth',None)
m=0
for i in range(0,len(doc_list)):
  finaldf=pd.DataFrame(columns=['paper_id','title','doi','text','score','answer_exercpt'])
  display(HTML('<h1> '+queries[i]+'</h1>'))
  k=0
  for id in doc_list[i]:
    for index,row in df_dict[id].T.iterrows():
      link=str(row['doi'])
      title=row["title"]
      linka='http://doi.org/'+link
      final_link=linka
      answer=answer_question2(queries[m],row['text'])
      to_append = [row['paper_id'],title,final_link,row['text'][:10],top10_scores_sorted[i][k],answer]
      df_length = len(finaldf)
      finaldf.loc[df_length] = to_append
      k+=1
  m+=1
  display(finaldf)

Token indices sequence length is longer than the specified maximum sequence length for this model (1774 > 512). Running this sequence through the model will result in indexing errors


,paper_id,title,doi,text,score,answer_exercpt
0,e86a2d96cb6bea7787c58496b384c59b788017dd,COVID-19: Global Health Equity in Pandemic Response,http://doi.org/10.4269/ajtmh.20-0260,COVID-19:,27.669937,Appropriate answer not found
1,e54cfdf6d59ae0174571e71ead0061a9b32b80c3,PPE or not PPE - that is the question,http://doi.org/10.1038/s41415-020-1639-y,PPE or not,25.250862,Appropriate answer not found
2,eb819f89014a6e662d0208c962155b91e91d6584,Singing Away the Social Distancing Blues: Art Therapy in a Time of Coronavirus,http://doi.org/10.1177/0022167820927807,Singing Aw,24.547355,Appropriate answer not found
3,ec59134c9675007a1de94a4fcce28c139fb1baeb,"The Mortal Coil of Covid-19, Fake News, and Negative Epistemic Postdigital Inculcation",http://doi.org/10.1007/s42438-020-00192-7,The Mortal,24.442839,"work with the rise of modern digital media , has primarily become visible because of the interrelationships between implicit learning , covid - 19 , fake news , and digital media . while the inculcation outlined in this paper occurs mostly outside of our awareness , i discuss a role for education in helping reduce the ensuing mortal coil of fake news . the mortal coil 1 of covid - 19pan"
4,f8c120006ddc1c256306a35759e00aaa505b9c6b,Coronavirus Pandemic as Black Swan Event,http://doi.org/10.1007/978-3-030-49264-9_32,Coronaviru,23.811713,recognized as black swan event according to considered figures and facts from recent references . introductionthere is a worldwide concern about the new coronavirus 2019 - ncov as a global public health threat . the 2019 - ncov infection is spreading and its incidence is increasing worldwide . although the current mortality is lower than that of the sars - cov and the mer
5,f4c762289a704d61cde8da3ddae21fc7a0ed5e99,Psychologist Self-Care During the Pandemic: Now More Than Ever,http://doi.org/10.1007/s42843-020-00010-5,Psychologi,23.799986,"during these extraordinary times . clinical vignettesgail hopkins , psyd , a 39 - year - old black psychologist , wakes up feeling anxious and falls asleep feeling anxious about the covid - 19 pandemic . in between , she experiences diffuse background worry most of the day . as a member of the sandwich generation , dr . hopkins worries about both her two young children quarantined at home and her aging parents with serious health problems . she juggles her half - time responsibilities at the local psychiatric hospital , where she continues to see patients in person , with her half - time position in a group private practice , where she sees all patients seamlessly via tel"
6,eef47e7531ebe9031aa62bd738e7cb36e9aade5e,Systematic review of international guidelines for tracheostomy in COVID-19 patients,http://doi.org/10.1016/j.oraloncology.2020.104844,Systematic,23.661664,Appropriate answer not found
7,eb98e3dd727cf16112cc9ed441c3a2105a332830,Pandemic COVID-19 Joins History’s Pandemic Legion,http://doi.org/10.1128/mbio.00812-20,Pandemic C,23.597627,"sekheperenre ramesses v clearly shows smallpox lesions ( fig . 2 ) , indicating that fatal smallpox epidemics prevailed more than 3 , 000 years ago . at some point , smallpox spread pandemically over most of the world , sparing the western hemisphere for millennia , up to the 16th century , when the first known epidemic occurred there in 1520 . until its eradication was declared in 1980 , smallpox killed untold millions over at least 3 millennia . even diseases now commonly dismissed as benign , like measles , killed millions ; little more than a century ago , me"
8,eae9dd3b8d09f5e255ab24631fafe10d203752b2,Pre-outbreak determinants of perceived risks of corona infection and preventive measures taken. A prospective population-based study,http://doi.org/10.1371/journal.pone.0234600,Pre-outbre,23.435152,Appropriate answer not found
9,edd549ddad8a46749e862be50560fb9aeb2573c1,Global health landscape challenges triggered by COVID-19,http://doi.org/10.1186/s41232-020-00144-5,Global hea,23.256059,Appropriate answer not found


,paper_id,title,doi,text,score,answer_exercpt
0,e26214244bdd92df2c0967ea3e3d0054bfe785f5,In Vitro Diagnostic Assays for COVID-19: Recent Advances and Emerging Trends,http://doi.org/10.3390/diagnostics10040202,In Vitro D,16.425396,Appropriate answer not found
1,76e3402f66d6e42dd7459aeb92773811ba094fa4,Effectiveness of COVID-19 diagnosis and management tools: A review,http://doi.org/10.1016/j.radi.2020.09.010,Effectiven,14.450643,"- pcr ) assay is considered the first tool to make a definitive diagnosis of covid - 19 disease , the high false negative rate , low sensitivity , limited supplies and strict requirements for laboratory settings might delay accurate diagnosis . computed tomography ( ct ) has been reported as an important tool to identify and investigate suspected patients with covid - 19 disease at early stage . findings rt - pcr shows low sensitivity ( 60 – 71 % ) in diagnosing patients with covid - 19 infection compared to the ct chest . several studies reported that chest ct scans show typical imaging features in all patients with covid - 19 . this high sensitivity and initial presentation in ct chest can be helpful in rectifying false negative results obtained from rt - pcr . as covid - 19 has similar manifestations to other pneumonia diseases , artificial intelligence ( ai ) might help radiologists to differentiate covid - 19 from other pneumonia diseases . conclusion although ct scan is a powerful tool in covid - 19 diagnosis , it is not sufficient to detect covid - 19 alone due to the low specificity ( 25 % ) , and challenges that radiologists might face in differentiating covid - 19 from other viral pneumonia on chest ct scans . ai might help radiologists to differentiate covid - 19 from other pneumonia diseases . implication for practice both rt - pc"
2,e2149d1c247ef118df46869e0e8822198f10ad23,The role of peripheral blood eosinophil counts in COVID‐19 patients,http://doi.org/10.1111/all.14465,The role o,14.263009,". 07 ) , later than that of body temperature ( 12days vs 10days"
3,e2930e630933e6fabddd00369308f8250a18a187,Fast Accurate Point of Care COVID-19 Pandemic Diagnosis Enabled Through Advanced Lab-on-a-Chip Optical Biosensors: Opportunities and Challenges,http://doi.org/nan,Fast Accur,14.193659,Appropriate answer not found
4,e046a06f262d74d025272ad697d358c4afb5d9bd,Application and optimization of RT-PCR in diagnosis of SARS-CoV-2 infection,http://doi.org/10.1101/2020.02.25.20027755,Applicatio,13.945084,Appropriate answer not found
5,e2988719c558bf688974fbd9478a1804439521fc,Navigating the Diagnostics of COVID-19,http://doi.org/10.1007/s42399-020-00408-8,Navigating,13.118135,". an electronic literature search was performed for peer - reviewed articles published from january 1 , 2020 , until june 19 , 2020 . published articles were then reviewed and included based on the applicability to the topic . the preferred diagnostic approach is the reverse transcription ( rt ) of the virus ’ ribonucleic acid ( rna ) followed by polymerase chain reaction ( pcr ) amplification ( rt - pcr ) . however , this method has been proven to be time - consuming . in improving the speed and efficiency of diagnostics , newer rapid diagnostic serological tests are in development for testing sars - cov - 2 , each with its unique advantages and disadvantages . they could potentially be used as triage tests to rapidly identify patients who are very likely to have covid - 19 in combination with other accurate diagnostic methods , such as diagnostic imaging . a combination of the disease history , clinical manifestations , laboratory diagnostic testing , and diagnostic imaging is crucial for making an accurate and useful diagnosis for covid - 19 . hopefully , the continuous development and use of rapid diagnostic tests and the implementation of public health measures will help control the spread of the disease . introductionthe coronavirus disease 2019 ( covid - 19 ) started in hubei province , china , and is caused by severe acute res

,paper_id,title,doi,text,score,answer_exercpt
0,e41ef8fb9ea0a8c6abd2159fd94d12a86267806d,Challenges for Microelectronics in Non-Invasive Medical Diagnostics,http://doi.org/10.3390/s20133636,Challenges,29.398444,Appropriate answer not found
1,e441a34d18665aa5b139544cdad74ba17d7d4a13,Direct diagnostic testing of SARS-CoV-2 without the need for prior RNA extraction.,http://doi.org/10.1101/2020.05.28.20115220,Direct dia,28.824963,"- to - implement , sars - cov - 2 test that utilizes isothermal amplification and can be run directly on viral transport media following a nasopharyngeal swab without the need for prior rna extraction . our assay provides visual results in 30 min with 85 % sensitivity , 100 % specificity , and a limit of detection ( lod ) of 2 . 5 copies / l , and can be run using a simple heat block . introductionthe coronavirus disease 2019 ( covid - 19 ) pandemic caused by sars - coronavirus 2 ( sar"
2,e391867b948f595cfe748251586ce599ad3e82ef,"Development and validation of a portable, point-of-care canine distemper virus qPCR test",http://doi.org/10.1371/journal.pone.0232044,Developmen,28.722656,Appropriate answer not found
3,f3c018d556cad261ea04fade08d60e5eea7c3859,Point-of-Care Diagnostic Tests for Detecting SARS-CoV-2 Antibodies: A Systematic Review and Meta-Analysis of Real-World Data,http://doi.org/10.3390/jcm9051515,Point-of-C,28.348844,"- of - care test and rapid bed - side tests [SEP] point - of - care diagnostic tests for detecting sars - cov - 2 antibodies : a systematic review and meta - analysis of real - world data sars - cov - 2 is responsible for a highly contagious infection , known as covid - 19 . sar"
4,e8972b4bfd839e811780fd9ecfc7262152a677ef,Hospital At Home Units In The Post‐Covid 19 ERA,http://doi.org/10.1111/eci.13390,Hospital A,27.705653,Appropriate answer not found
5,e775ae09a7e0906313852000f0700011a0415682,A proposal for the return to routine endoscopy during the COVID-19 pandemic,http://doi.org/10.1016/j.gie.2020.04.050,A proposal,27.580749,Appropriate answer not found
6,ee024840b40e3086ed1d6feb1134fd7d762edc00,COVID-19 paraclinical diagnostic tools: Updates and future trends,http://doi.org/10.1016/j.retram.2020.06.001,COVID-19 p,27.293134,"other , coupling efficiency and less costly procedures may permit more effective community - scale management . methodology and main structure : in this review , we shed light on the most used and the most validated diagnostic tools . furthermore , we intend to include few under - development techniques that may be potentially useful in this context . the practical intent of our work is to provide clinicians with a realistic summarized review of the essential elements in the applied paraclinical diagnosis of covid - 19 . introductionin december 2019 , an increasing number of pneumonia cases appeared in wuhan , hubei state in china . thorough analytical studies , accomplished by epidemiologists , demonstrated that the spread of the disease might be related to wuhan south china seafood market . dedicated in - depth studies , using highthroughput sequencing , revealed a new beta - coronavirus that was called 2019 novel coronavirus ( 2019 - ncov ) . in january 2020 , the world health organization ( who ) renamed the virus as sar"
7,f7cd33f182af02e02408bf4af35b5aad2eddf645,Antibody tests in detecting SARS-CoV-2 infection: a meta-analysis,http://doi.org/10.1101/2020.04.22.20074914,Antibody t,27.002349,Appropriate answer not found
8,e33791175d69e3d900ed203acdb00327a1fc96f3,"An easy, reliable and rapid SARS-CoV2 RT-LAMP based test for Point-of-Care and diagnostic lab",http://doi.org/10.1101/2020.09.25.20200956,"An easy, r",26.993584,"- of - care format uses loopx ( c ) , a small portative device ensuring optimal lamp reaction and automated reading with 95 . 2 % and 95 . 5 % sensitivity and specificity respectively . this system might also be useful for testing other sample types such as saliva . rt - lamp ( reverse - transcription and loop - mediated isothermal a

,paper_id,title,doi,text,score,answer_exercpt
0,edb4f309c40054108af5620644f06c68c68b0768,Analysis of Spatial-temporal Behavior Pattern of the Share Bike Usage during COVID-19 Pandemic in Beijing,http://doi.org/nan,Analysis o,20.211438,Appropriate answer not found
1,f7cbc45f7e4be98a98fc0f9546fa60c25ebbcfda,Isothermal Nucleic Acid Amplification Techniques and Their Use in Bioanalysis,http://doi.org/10.1134/s0006297920020030,Isothermal,17.507062,"describes the principles of isothermal amplification techniques and demonstrates their high efficiency in designing new highly sensitive detection methods of nucleic acids and enzymes involved in their modifications . the data on successful application of isothermal amplification methods for the analysis of cells and biomolecules with the use of dna / rna aptamers are presented . quantitative and qualitative determination of nucle ic acids is an important problem of modern biology and medicine . this research area has been developing very rapidly since the early 1990s , and it is very likely that the number of studies on this topic will continue to grow in the next decades . detection of dna / rna of pathogenic bacteria and viruses could be essential for choosing an appropriate treatment strategy . recent studies have dis covered the correlation between the risk of developing certain diseases in humans and single nucleotide poly morphisms or short insertions / deletions . it was also found that human microrna genes are often located in the vicinity of genome regions and sites associated with cancer . the expression levels of some micrornas in patients with chronic"
2,e673817674626d033b5b9c861584bd42da1e5c50,"High Tech, High Risk: Tech Ethics Lessons for the COVID-19 Pandemic Response",http://doi.org/10.1016/j.patter.2020.100102,"High Tech,",15.974420,Appropriate answer not found
3,76e34b0b3e937b1062f12fccbaf924503e916bc8,Combined drug repurposing and virtual screening strategies with molecular dynamics simulation identified potent inhibitors for SARS-CoV-2 main protease (3CLpro),http://doi.org/10.1080/07391102.2020.1779128,Combined d,15.477545,Appropriate answer not found
4,f0acd81bca3e22cb8a3b480d16437c0f72d46c41,"Lions, Tigers and Kittens too: ACE2 and susceptibility to CoVID-19",http://doi.org/10.1093/emph/eoaa021,"Lions, Tig",14.832875,Appropriate answer not found
5,e14f69047a759c01eb0f61c592203181506fb723,In Silico Screening of Some Naturally Occurring Bioactive Compounds Predicts Potential Inhibitors against SARS-COV-2 (COVID-19) Protease,http://doi.org/nan,In Silico,14.828693,still lacking . we firmly believe that nature itself provides a simple solution for any complicated problem created in it which motivated us to carry out in silico investigations on some bioactive natural compounds reportedly found in the fruits and leaves of anthocephalus cadamba which is a miraculous plant found on the earth aiming to predict the potential inhibitors against afores
6,69f5445e541847993640ec68027ed73ee1980aa5,Trust and Transparency in Contact Tracing Applications,http://doi.org/nan,Trust and,14.738156,"given the exponential spread of the virus that causes covid - 19 , there has been significant interest in the development and use of digital contact tracing solutions to supplement the work of human contact tracers . the collection and use of sensitive personal details by these applications has led to a number of concerns by the stakeholder groups with a vested interest in these solutions . we explore digital contact tracing solutions in detail and propose the use of a transparent reporting mechanism , factsheets , to provide transparency of and support trust in these applications . we also provide an example factsheet template with questions that are specific to the contact tracing application domain . abstract - the global outbreak of covid - 19 has led to focus on efforts to manage and mitigate the continued spread of the disease . one of these efforts include the use of contact tracing to i

,paper_id,title,doi,text,score,answer_exercpt
0,e84dc78d7df6a96279d846c93f7921763834fe87,"An effective, safe and cost-effective cell-based chimeric vaccine against SARS-CoV2",http://doi.org/10.1101/2020.08.19.258244,An effecti,13.663271,Appropriate answer not found
1,e35d6c05896a75cddc4c93d3fb6e7eac55087a95,Detectable severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) in human breast milk of a mildly symptomatic patient with coronavirus disease 2019 (COVID-19),http://doi.org/10.1093/cid/ciaa673,Detectable,13.238074,Appropriate answer not found
2,eae0effca94e696cb437cf256e70428b3dd02e29,Mapping the Global Research and Clinical Trials in COVID-19,http://doi.org/10.1101/2020.06.27.20141788,Mapping th,13.235497,Appropriate answer not found
3,e83cff607fc35bff49974c0f7283ee1a7e349704,Artificial intelligence predicts the immunogenic landscape of SARS-CoV-2: toward universal blueprints for vaccine designs,http://doi.org/10.1101/2020.04.21.052084,Artificial,13.223823,"present suffering from a pandemic of coronavirus disease 2019 ( covid - 19 ) , caused by the novel coronavirus severe acute respiratory syndrome coronavirus 2 ( sars - cov - 2 ) . the goals of this study were to use artificial intelligence ( ai ) to predict blueprints for designing universal vaccines against sars - cov - 2 , that contain a sufficiently broad repertoire of t - cell epitopes capable of providing coverage and protection across the global population . to help achieve these aims , we profiled the entire sars - cov - 2 proteome across the most frequent 100 hla - a , hla - b and hla - dr alleles in the human population , using host - infected cell surface antigen presentation and immunogenicity predictors from the nec immune profiler suite of tools , and generated comprehensive epitope maps . we then used these epitope maps as input for a monte carlo simulation designed to identify statistically significant “ epitope hotspot ” regions in the virus that are most likely to be immunogenic across a broad spectrum of hla types . we then removed epitope hotspots that shared significant homology with proteins in the human proteome to reduce the chance of inducing off - target autoimmune responses . we also analyzed the antigen presentation and immunogenic landscape of all the nonsynonymous mutations across 3400 different sequences of the virus , to identify a trend whereby sars - cov - 2 mutations are predicted to have reduced potential to be presented by host - infected cells , and consequently detected by the host immune system . a sequence conservation analysis then removed epitope hotspots that occurred in less - conserved regions of the viral proteome . finally , we used a database of the hla genotypes of approximately 22 000 individuals to develop a “ digital twin"
4,e421194b230d5a0a59ea097587a76da65e3e0b3b,Clinical characteristics and humoral immune response in Healthcare Workers with COVID-19 in a Teaching Hospital in Belgium,http://doi.org/10.1016/j.jhin.2020.09.018,Clinical c,12.868182,Appropriate answer not found
5,e69e83feb23c448fd846dcb17a577ea3024442ab,Profile of specific antibodies to SARS-CoV-2: The first report,http://doi.org/10.1016/j.jinf.2020.03.012,Profile of,12.816554,Appropriate answer not found
6,f2ca707d0b5e2eb201ca53ca919b763b2fd8ad9e,"Cerebellar Infarction Requiring Surgical Decompression in patient with COVID 19 Pathological Analysis, Brief Review",http://doi.org/10.1016/j.inat.2020.100850,Cerebellar,12.802594,"given subsequent rapid neurologic decline from cerebellar swelling and mass effect on his brainstem emergent neurosurgical intervention was performed . brain biopsy found a vein with small organizing thrombus adjacent to focally proliferative intima with focal intimal neutrophils . conclusion : a young man with covid - 19 and suspected immune dysregulation , complicated by a large cerebrovascular ischemic stroke secondary to vertebral artery thrombosis requiring emergent neurosurgical intervention for decompression with impr

,paper_id,title,doi,text,score,answer_exercpt
0,ebadaaaf1d5948fae5ecf78a61d18a7db3e0ca4b,A Minimalist Strategy Towards Temporarily Defining Protection for COVID-19,http://doi.org/10.1007/s42399-020-00533-4,A Minimali,42.445346,"while yet controlling the pandemic with current approaches . for those infected thus far , there is a prevailing perspective that devising recognition for protective immunity will progressively allow segments of society to return to some functionality more than is existing . at this time , the best correlates with protection from natural coronavirus infections are systemic"
1,71a10e9373bea71c5a30cc9de81c794764d63ebc,Practical Aspects and Considerations When Planning a New Clinical Microbiology Laboratory,http://doi.org/10.1016/j.cll.2020.08.015,Practical,40.955022,meet growing and changing needs will extend the lifetime of the space . open floor plans may challenge some sensibilities and biosafety concerns but are the vogue for bs
2,77ad3841ab962b29f5ef7af6c72e69845b7f1805,Molecular Testing for Acute Respiratory Tract Infections: Clinical and Diagnostic Recommendations From the IDSA’s Diagnostics Committee,http://doi.org/10.1093/cid/ciaa508,Molecular,40.066180,"given the plethora of available assays and rapidly evolving technologies . here , we summarize the current state of the art with respect to the diagnosis of viral and bacterial rtis , provide a practical framework for diagnostic decision making using selected patient - centered vignettes , and make recommendations for future studies to advance the field . the clinical signs and symptoms of acute respiratory tract infections ( rtis ) are not pathogen specific . highly sensitive and specific nucleic acid amplification tests have become the diagnostic reference standard for viruses , and translation of bacterial ass"
3,77b1e88254e4aa07d5841d2c9a09de09bf6069d3,"Associations between psychiatric disorders, COVID-19 testing probability and COVID-19 testing results: findings from a population-based study",http://doi.org/10.1192/bjo.2020.75,Associatio,39.316024,"aims : to compare covid - 19 testing probability and results among individuals with psychiatric disorders with those without such diagnoses , and to examine the associations between testing probability and results and psychiatric diagnoses . method : this is a population - based study to perform association analyses of psychiatric disorder diagnoses with covid - 19 testing probability and such test results , by using two - sided fisher exact tests and logistic regression . the population were uk biobank participants who had undergone covid - 19 testing . the main outcomes were covid - 19 testing probability and covid - 19 test results . results : individuals with psychiatric disorders were overrepresented among the 1474 uk biobank participants with test data : 23 % of the covid - 19 test sample had a psychiatric diagnosis compared with 10 % in the full cohort ( p < 0 . 0001 ) . this overrepresentation persisted for each of the specific psychiatric disorders tested . furthermore , individuals with a psychiatric disorder ( p = 0 . 01 ) , particularly substance use disorder ( p < 0 . 005 ) , had negative test results significantly more often than individuals without psychiatric disorders . sensitivity analyses confirmed our results . conclusions : in contrast with our hypotheses , uk biobank participants with psychiatric disorders have been tested for covid - 19 more frequently than individuals without a psychiatric history . among those tested , test outcomes were more frequently negative for registry participants with psychiatric disorders than in others , countering arguments that people with psychiatric disorders are particularly prone to contract the virus . backgroundcoronavirus disease 2019 caused by the novel sars - co"
4,70b66d624b96569304621663178b5759e3dd1d3f,The Epistemology of a Positive SARS-CoV-2 Test,http://doi.org/10.1007/s10441-020-09393-w,The Episte,39.036207,"is the hypothesis tha

# Model 5:
# Result obtained from bert-base-cased-finetuned-mrpc model has minimum missing answer excerpts

In [ ]:
from IPython.core.display import display, HTML

pd.set_option('display.max_colwidth',None)
m=0
for i in range(0,len(doc_list)):
  finaldf=pd.DataFrame(columns=['paper_id','title','doi','text','score','answer_exercpt'])
  display(HTML('<h1> '+queries[i]+'</h1>'))
  k=0
  for id in doc_list[i]:
    for index,row in df_dict[id].T.iterrows():
      link=str(row['doi'])
      title=row["title"]
      linka='http://doi.org/'+link
      final_link=linka
      answer=answer_question5(queries[m],row['text'])
      to_append = [row['paper_id'],title,final_link,row['text'][:10],top10_scores_sorted[i][k],answer]
      df_length = len(finaldf)
      finaldf.loc[df_length] = to_append
      k+=1
  m+=1
  display(finaldf)


Token indices sequence length is longer than the specified maximum sequence length for this model (1844 > 512). Running this sequence through the model will result in indexing errors


,paper_id,title,doi,text,score,answer_exercpt
0,e86a2d96cb6bea7787c58496b384c59b788017dd,COVID-19: Global Health Equity in Pandemic Response,http://doi.org/10.4269/ajtmh.20-0260,COVID-19:,27.669937,"##agnostics , pharmaceutical interventions , and public health solutions for other pandemic diseases such as tuberculosis , HIV infection , and cholera have never fully "" trickled down "" to marginalized or impoverished communities . That is why , for"
1,e54cfdf6d59ae0174571e71ead0061a9b32b80c3,PPE or not PPE - that is the question,http://doi.org/10.1038/s41415-020-1639-y,PPE or not,25.250862,"( PPE ) , which makes patient - centred care ethically more challenging . Our first response to these ethical challenges should be to start with ourselves to make sure we are safe before we think about the patients . This is not the approach we would normally adopt when treating our patients outside of a pandemic . My colleague is telephone triaging patients during the lockdown phase of the COVID - 19 pandemic . No routine treatment is being provided in accordance with the current guidelines , and face - to - face consultations and treatments are being reduced to absolute necessity . 1 A 76 - year - old lady phones . She has been with the practice , regularly attending for a number of years . She lives on her own , has been self - isolating for a number of weeks and is in pain . The lower right third molar is carious and split , and it needs an extraction . She is on bisp"
2,eb819f89014a6e662d0208c962155b91e91d6584,Singing Away the Social Distancing Blues: Art Therapy in a Time of Coronavirus,http://doi.org/10.1177/0022167820927807,Singing Aw,24.547355,participate
3,ec59134c9675007a1de94a4fcce28c139fb1baeb,"The Mortal Coil of Covid-19, Fake News, and Negative Epistemic Postdigital Inculcation",http://doi.org/10.1007/s42438-020-00192-7,The Mortal,24.442839,"ability of humans to discern between fact and fiction diminishes . It has resulted in some people making life - ending decisions based on their exposure to fake news . In this article , I define a primarily ignored and invisible epistemological process at work : negative epistemic postdigital inculcation , that , while has been at work with the rise of modern digital media , has primarily become visible because of the interrel"
4,f8c120006ddc1c256306a35759e00aaa505b9c6b,Coronavirus Pandemic as Black Swan Event,http://doi.org/10.1007/978-3-030-49264-9_32,Coronaviru,23.811713,"according to considered figures and facts from recent references . IntroductionThere is a worldwide concern about the new coronavirus 2019 - nCoV as a global public health threat . The 2019 - nCoV infection is spreading and its incidence is increasing worldwide . Although the current mortality is lower than that of the SARS - CoV and the MERS - CoV , it seems that the 2019 - nC"
5,f4c762289a704d61cde8da3ddae21fc7a0ed5e99,Psychologist Self-Care During the Pandemic: Now More Than Ever,http://doi.org/10.1007/s42843-020-00010-5,Psychologi,23.799986,"practices of psychologists and has highlighted the need for time - efficient self - care . The anxiety among psychologists ( and humans ) is practically universal — nearly every psychologist worldwide is experiencing some negative impact on their mental health . We offer nine research - supported , practitioner - friendly methods of self - care to manage anxiety and maintain optimal performance during these extraordinary times . Clinical VignettesGail Hopkins , PsyD , a 39 - year - old Black psychologist , wakes up feeling anxious and falls asleep feeling anxious about the COVID - 19 pandemic . In between , she experiences diffuse background worry most of the day . As a member of the sandwich generation , Dr . Hopkins worries about both her two young children quarantined at home and her aging parents with serious health problems . She juggles her half - time responsibilities at the local psychiatric hospital , where she continues to see patients in person , with her half - time

,paper_id,title,doi,text,score,answer_exercpt
0,e26214244bdd92df2c0967ea3e3d0054bfe785f5,In Vitro Diagnostic Assays for COVID-19: Recent Advances and Emerging Trends,http://doi.org/10.3390/diagnostics10040202,In Vitro D,16.425396,"( EUA ) and are Conformité Européenne ( CE ) marked . A wide range of serology immunoassays ( IAs ) have also been developed that complement the molecular assays for the diagnosis of COVID - 19 . The most prominent IAs are automated chemiluminescent IA ( CLIA ) , manual ELISA , and rapid lateral flow IA ( LFIA ) , which detect the immunoglobulin M ( IgM ) and immuno"
1,76e3402f66d6e42dd7459aeb92773811ba094fa4,Effectiveness of COVID-19 diagnosis and management tools: A review,http://doi.org/10.1016/j.radi.2020.09.010,Effectiven,14.450643,Appropriate answer not found
2,e2149d1c247ef118df46869e0e8822198f10ad23,The role of peripheral blood eosinophil counts in COVID‐19 patients,http://doi.org/10.1111/all.14465,The role o,14.263009,"EOS counts , which was significantly more frequent than other types of pneumonia patients . EOS counts had good value for COVID ‐ 19 prediction , even higher when combined with NLR . Patients with low EOS counts at admission were more likely to have fever , fatigue and shortness of breath , with more lesions in chest CT and radiographic aggravation , longer length of hospital stay and course of disease than those with normal EOS counts . Circulating EOS level gradually increased over the time , and was synchronous with the improvement of chest CT ( 12days vs 13days , P = 0 . 07 ) , later than that of body temperature ( 12days vs 10days , P = 0 . 014 ) , but was earlier than the negative conversion of nucleic"
3,e2930e630933e6fabddd00369308f8250a18a187,Fast Accurate Point of Care COVID-19 Pandemic Diagnosis Enabled Through Advanced Lab-on-a-Chip Optical Biosensors: Opportunities and Challenges,http://doi.org/nan,Fast Accur,14.193659,"1 fM concentration along with few minutes sensing . These biosensors can be manufactured on a mass - scale ( billions ) to detect the COVID - 19 viral load in nasal , saliva , urinal , and serological samples even if the infected person is asymptotic . Methods investigated here are the most advanced available platforms for biosensing optical devices resulted from the integration of state - of - the - art designs and materials . These approaches are including but not limited to integrated optical devices , plasmonic resonance and also emerging nanomaterial biosensors . The lab - on"
4,e046a06f262d74d025272ad697d358c4afb5d9bd,Application and optimization of RT-PCR in diagnosis of SARS-CoV-2 infection,http://doi.org/10.1101/2020.02.25.20027755,Applicatio,13.945084,5 % of which ( 20 / 32 ) showed positive RT - PC
5,e2988719c558bf688974fbd9478a1804439521fc,Navigating the Diagnostics of COVID-19,http://doi.org/10.1007/s42399-020-00408-8,Navigating,13.118135,ribonucleic acid ( RNA ) followed by polymerase chain reaction ( PCR ) amp
6,76e34b0b3e937b1062f12fccbaf924503e916bc8,Combined drug repurposing and virtual screening strategies with molecular dynamics simulation identified potent inhibitors for SARS-CoV-2 main protease (3CLpro),http://doi.org/10.1080/07391102.2020.1779128,Combined d,13.046301,"##4 , 653 confirmed cases and 302 , 169 deaths are reported . The growing infection rate and death toll demand the use of all possible approaches to design novel drugs and vaccines to curb this disease . In this study , we combined drugs repurposing and virtual drug screening strategies to target 3CLpro , which has an essential role in viral maturation and replication . A total of 31 FDA approved anti - HIV drugs , and Traditional Chinese medicines ( TCM ) database were screened to find potential inhibitors . As a result , Saquinavir , and five drugs ( TCM5280805 , TCM5280445 , TCM5280343 , TCM"
7,f0acd81bca3e22cb8a3b480d16437c0f72d46c41,"Lions, Tigers and Kittens too: ACE2 and susceptibility to CoVID-19",http://doi.org/10.1093/emph/eoaa021,"Lions, Tig",12.92767

,paper_id,title,doi,text,score,answer_exercpt
0,e41ef8fb9ea0a8c6abd2159fd94d12a86267806d,Challenges for Microelectronics in Non-Invasive Medical Diagnostics,http://doi.org/10.3390/s20133636,Challenges,29.398444,"( MRI ) . The role of integrated circuits is central in both application domains . In portable analytical platforms , ASICs offer miniaturization and tackle the noise / power dissipation trade - off . The integration of CMOS chips with microfluidics poses multiple open technological issues . In multi - modal imaging , now that the compatibility of the acquisition chains ( thousands of Silicon Photo - Multip"
1,e441a34d18665aa5b139544cdad74ba17d7d4a13,Direct diagnostic testing of SARS-CoV-2 without the need for prior RNA extraction.,http://doi.org/10.1101/2020.05.28.20115220,Direct dia,28.824963,Appropriate answer not found
2,e391867b948f595cfe748251586ce599ad3e82ef,"Development and validation of a portable, point-of-care canine distemper virus qPCR test",http://doi.org/10.1371/journal.pone.0232044,Developmen,28.722656,Appropriate answer not found
3,f3c018d556cad261ea04fade08d60e5eea7c3859,Point-of-Care Diagnostic Tests for Detecting SARS-CoV-2 Antibodies: A Systematic Review and Meta-Analysis of Real-World Data,http://doi.org/10.3390/jcm9051515,Point-of-C,28.348844,Appropriate answer not found
4,e8972b4bfd839e811780fd9ecfc7262152a677ef,Hospital At Home Units In The Post‐Covid 19 ERA,http://doi.org/10.1111/eci.13390,Hospital A,27.705653,Appropriate answer not found
5,e775ae09a7e0906313852000f0700011a0415682,A proposal for the return to routine endoscopy during the COVID-19 pandemic,http://doi.org/10.1016/j.gie.2020.04.050,A proposal,27.580749,Appropriate answer not found
6,ee024840b40e3086ed1d6feb1134fd7d762edc00,COVID-19 paraclinical diagnostic tools: Updates and future trends,http://doi.org/10.1016/j.retram.2020.06.001,COVID-19 p,27.293134,Appropriate answer not found
7,f7cd33f182af02e02408bf4af35b5aad2eddf645,Antibody tests in detecting SARS-CoV-2 infection: a meta-analysis,http://doi.org/10.1101/2020.04.22.20074914,Antibody t,27.002349,Appropriate answer not found
8,e33791175d69e3d900ed203acdb00327a1fc96f3,"An easy, reliable and rapid SARS-CoV2 RT-LAMP based test for Point-of-Care and diagnostic lab",http://doi.org/10.1101/2020.09.25.20200956,"An easy, r",26.993584,LAMP ( Reverse - Transcription and Loop - mediated isothermal AMPlification ) might advantageously increase testing efficiency as it allows all - in
9,e26214244bdd92df2c0967ea3e3d0054bfe785f5,In Vitro Diagnostic Assays for COVID-19: Recent Advances and Emerging Trends,http://doi.org/10.3390/diagnostics10040202,In Vitro D,26.962000,"( EUA ) and are Conformité Européenne ( CE ) marked . A wide range of serology immunoassays ( IAs ) have also been developed that complement the molecular assays for the diagnosis of COVID - 19 . The most prominent IAs are automated chemiluminescent IA ( CLIA ) , manual ELISA , and rapid lateral flow IA ( LFIA ) , which detect the immunoglobulin M ( IgM ) and immuno"


,paper_id,title,doi,text,score,answer_exercpt
0,edb4f309c40054108af5620644f06c68c68b0768,Analysis of Spatial-temporal Behavior Pattern of the Share Bike Usage during COVID-19 Pandemic in Beijing,http://doi.org/nan,Analysis o,20.211438,Appropriate answer not found
1,f7cbc45f7e4be98a98fc0f9546fa60c25ebbcfda,Isothermal Nucleic Acid Amplification Techniques and Their Use in Bioanalysis,http://doi.org/10.1134/s0006297920020030,Isothermal,17.507062,"RNA aptamers are presented . Quantitative and qualitative determination of nucle ic acids is an important problem of modern biology and medicine . This research area has been developing very rapidly since the early 1990s , and it is very likely that the number of studies on this topic will continue to grow in the next decades . Detection of DNA / RNA of pathogenic bacteria and viruses could be essential for choosing an appropriate treatment strategy . Recent studies have dis covered the correlation between the risk of developing certain diseases in humans and single nucleotide poly morphisms or short insertions / deletions . It was also found that human microRNA genes are often located in the vicinity of genome regions and sites associated with cancer . The expression levels of some microRNAs in patients with chronic lymphocytic leukemia , colorectal neop"
2,e673817674626d033b5b9c861584bd42da1e5c50,"High Tech, High Risk: Tech Ethics Lessons for the COVID-19 Pandemic Response",http://doi.org/10.1016/j.patter.2020.100102,"High Tech,",15.974420,Appropriate answer not found
3,76e34b0b3e937b1062f12fccbaf924503e916bc8,Combined drug repurposing and virtual screening strategies with molecular dynamics simulation identified potent inhibitors for SARS-CoV-2 main protease (3CLpro),http://doi.org/10.1080/07391102.2020.1779128,Combined d,15.477545,Appropriate answer not found
4,f0acd81bca3e22cb8a3b480d16437c0f72d46c41,"Lions, Tigers and Kittens too: ACE2 and susceptibility to CoVID-19",http://doi.org/10.1093/emph/eoaa021,"Lions, Tig",14.832875,evolution - based approaches can be used to form hypot
5,e14f69047a759c01eb0f61c592203181506fb723,In Silico Screening of Some Naturally Occurring Bioactive Compounds Predicts Potential Inhibitors against SARS-COV-2 (COVID-19) Protease,http://doi.org/nan,In Silico,14.828693,Appropriate answer not found
6,69f5445e541847993640ec68027ed73ee1980aa5,Trust and Transparency in Contact Tracing Applications,http://doi.org/nan,Trust and,14.738156,"FactSheets , to provide transparency of and support trust in these applications . We also provide an example FactSheet template with questions that are specific to the contact tracing application domain . Abstract - The global outbreak of COVID - 19 has led to focus on efforts to manage and mitigate"
7,e2930e630933e6fabddd00369308f8250a18a187,Fast Accurate Point of Care COVID-19 Pandemic Diagnosis Enabled Through Advanced Lab-on-a-Chip Optical Biosensors: Opportunities and Challenges,http://doi.org/nan,Fast Accur,14.723498,"1 fM concentration along with few minutes sensing . These biosensors can be manufactured on a mass - scale ( billions ) to detect the COVID - 19 viral load in nasal , saliva , urinal , and serological samples even if the infected person is asymptotic . Methods investigated here are the most advanced available platforms for biosensing optical devices resulted from the integration of state - of - the - art designs and materials . These approaches are including but not limited to integrated optical devices , plasmonic resonance and also emerging nanomaterial biosensors . The lab - on"
8,e83cff607fc35bff49974c0f7283ee1a7e349704,Artificial intelligence predicts the immunogenic landscape of SARS-CoV-2: toward universal blueprints for vaccine designs,http://doi.org/10.1101/2020.04.21.052084,Artificial,14.549002,Appropriate answer not found
9,78bcf659c7764369a3044e829791bd4e42e3d559,Contact Tracing Mobile Apps for COVID-19: Privacy Considerations and Related Trade-offs,http://doi.org/nan,Contact Tr,14.410828,"is iden

,paper_id,title,doi,text,score,answer_exercpt
0,e84dc78d7df6a96279d846c93f7921763834fe87,"An effective, safe and cost-effective cell-based chimeric vaccine against SARS-CoV2",http://doi.org/10.1101/2020.08.19.258244,An effecti,13.663271,"to its receptor . Similar to SARS - CoV , SARS - CoV - 2 S protein also adopts human angiotens"
1,e35d6c05896a75cddc4c93d3fb6e7eac55087a95,Detectable severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) in human breast milk of a mildly symptomatic patient with coronavirus disease 2019 (COVID-19),http://doi.org/10.1093/cid/ciaa673,Detectable,13.238074,Appropriate answer not found
2,eae0effca94e696cb437cf256e70428b3dd02e29,Mapping the Global Research and Clinical Trials in COVID-19,http://doi.org/10.1101/2020.06.27.20141788,Mapping th,13.235497,Appropriate answer not found
3,e83cff607fc35bff49974c0f7283ee1a7e349704,Artificial intelligence predicts the immunogenic landscape of SARS-CoV-2: toward universal blueprints for vaccine designs,http://doi.org/10.1101/2020.04.21.052084,Artificial,13.223823,##mology with proteins in the human proteome to reduce the chance of inducing off - target autoimmune responses . We also analyzed the antigen presentation and immuno
4,e421194b230d5a0a59ea097587a76da65e3e0b3b,Clinical characteristics and humoral immune response in Healthcare Workers with COVID-19 in a Teaching Hospital in Belgium,http://doi.org/10.1016/j.jhin.2020.09.018,Clinical c,12.868182,Appropriate answer not found
5,e69e83feb23c448fd846dcb17a577ea3024442ab,Profile of specific antibodies to SARS-CoV-2: The first report,http://doi.org/10.1016/j.jinf.2020.03.012,Profile of,12.816554,after onset of symptoms to detect the specific antibodies to SARS - CoV - 2 . IgM and IgG were analyzed by chemilumines
6,f2ca707d0b5e2eb201ca53ca919b763b2fd8ad9e,"Cerebellar Infarction Requiring Surgical Decompression in patient with COVID 19 Pathological Analysis, Brief Review",http://doi.org/10.1016/j.inat.2020.100850,Cerebellar,12.802594,"BACKGROUND : This report and literature review describes a case of a COVID - 19 patient who suffered a cerebellar stroke requiring neurosurgical decompression . This is the first reported case of a sub - occipital craniectomy with brain biopsy in a COVID - 19 patient showing leptomeningeal venous intimal inflammation . CLINICAL DESCRIPTION : The patient is a 48 - year - old SARS - COV - 2 positive male with multiple comorbidities , who presented with fevers and respiratory symptoms , and imaging consistent with multifocal pneumonia . On day 5 of admission , the patient had sudden change in mental status , increased C - Reactive Protein , ferritin and elevated Interleukin - 6 levels . Head CT showed cerebral infarction from vertebral artery occlusion . Given subsequent rapid neurologic decline from cerebellar swelling and mass effect on his brainstem emergent neurosurgical intervention was performed . Brain biopsy found a vein with small organizing thrombus adjacent to focally proliferative intima with focal intimal neutrophils . CONCLUSION : A young man with COVID - 19 and suspected immune dysregulation , complicated by a large cerebrovascular ischemic stroke secondary to vertebral artery thrombosis requiring emergent neurosurgical intervention for decompression with improved neuro"
7,e90a11bbc7b9cec4e674a515fbe317549f022be8,First results from the UK COVID-19 Serology in Oncology Staff Study (CSOS),http://doi.org/10.1101/2020.06.22.20136838,First resu,12.800850,Appropriate answer not found
8,ec04d705e885876d595e4f3df639173ddcb90f03,Facing SARS-CoV-2 outbreak in immunotherapy era,http://doi.org/10.2217/fon-2020-0340,Facing SAR,12.458333,##notherapy treatments in clinical practice . Severe acute respiratory syndrome coronavirus 2 ( SARS - CoV - 2 ) spread represents a sanitary emergency all over the world . Viral biology is only partially known with some aspects in common with other CoV and the damage observed in most severe cases is due to intense inflammation . Immunotherapy 

,paper_id,title,doi,text,score,answer_exercpt
0,ebadaaaf1d5948fae5ecf78a61d18a7db3e0ca4b,A Minimalist Strategy Towards Temporarily Defining Protection for COVID-19,http://doi.org/10.1007/s42399-020-00533-4,A Minimali,42.445346,"is achieved , there will be a need to regain world economic stability while yet controlling the pandemic with current approaches . For those infected thus far , there is a prevailing perspective that devising recognition for protective immunity will progressively allow segments of society to return to some functionality more than is existing . At this time , the best correlates with protection from natural coronavirus infections are systemic neutralizing antibody and mucosal IgA . Serum neutralizing antibody more easily fulfills the latter requisite , but current live virus methods for neutralization prevent large - scale application . It is conceivable that the exposure of previously infected individuals can allow for the definition of protective thresholds of neutralizing antibody . Thereafter , many other antibody assays will be able to screen for surrogate protection after correlations with protective neutralizing antibody are made . Specificity of common antibody tests would benefit from confirmatory blocking systems or confirmatory immu"
1,71a10e9373bea71c5a30cc9de81c794764d63ebc,Practical Aspects and Considerations When Planning a New Clinical Microbiology Laboratory,http://doi.org/10.1016/j.cll.2020.08.015,Practical,40.955022,Appropriate answer not found
2,77ad3841ab962b29f5ef7af6c72e69845b7f1805,Molecular Testing for Acute Respiratory Tract Infections: Clinical and Diagnostic Recommendations From the IDSA’s Diagnostics Committee,http://doi.org/10.1093/cid/ciaa508,Molecular,40.066180,Appropriate answer not found
3,77b1e88254e4aa07d5841d2c9a09de09bf6069d3,"Associations between psychiatric disorders, COVID-19 testing probability and COVID-19 testing results: findings from a population-based study",http://doi.org/10.1192/bjo.2020.75,Associatio,39.316024,Appropriate answer not found
4,70b66d624b96569304621663178b5759e3dd1d3f,The Epistemology of a Positive SARS-CoV-2 Test,http://doi.org/10.1007/s10441-020-09393-w,The Episte,39.036207,Appropriate answer not found
5,ead4eb0d9efefd60ae894fc2f3e685f1d47128bb,COVID-19 AND THE GLOBAL IMPACT ON COLORECTAL PRACTICE AND SURGERY,http://doi.org/10.1016/j.clcc.2020.05.011,COVID-19 A,38.394965,Appropriate answer not found
6,e42c5bb6d800efa745214caa8fc413c87f8984bd,"Leishmaniasis diagnosis: an update on the use of parasitological, immunological and molecular methods",http://doi.org/10.1007/s12639-020-01212-w,Leishmania,38.370999,Appropriate answer not found
7,f8e259c6559c2d79f12c98483bd2e340639cf549,Testing for SARS-CoV-2 (COVID-19): a systematic review and clinical guide to molecular and serological in-vitro diagnostic assays,http://doi.org/10.1016/j.rbmo.2020.06.001,Testing fo,38.115811,Appropriate answer not found
8,80c0361651ecf3fed3e4e08b6f6cb358389037b2,The Status Dynamics of Role Blurring in the Time of COVID-19,http://doi.org/10.1177/2378023120944358,The Status,38.053427,"##lavin and Schieman 2012 ; Voydanoff 2005 ) . According to Desrochers , Hilton , and Lar"
9,e528052f7dfa598b11d4a0402607cbc514877724,Evaluation of machine learning algorithms for Health and Wellness applications: a tutorial,http://doi.org/nan,Evaluation,37.975280,"of this tutorial to provide practical guidance on how to assess performance reliably and efficiently and avoid common traps . Instead of giving a list of do ' s and don ' t s , this tutorial tries to build a better understanding behind these do ' s and don ' t s and presents both the most relevant performance evaluation criteria as well as how to compute them . Along the way , we will indicate common mistakes and provide references discussing various topics more in - depth . IntroductionData - driven approaches for healthcare decision support , such as those driven by Machine Learning ( ML ) , have seen a surge in interest over

In [ ]:
!pip install cdqa

     |████████████████████████████████| 51kB 3.2MB/s 
     |████████████████████████████████| 102kB 5.7MB/s 
     |████████████████████████████████| 286kB 31.2MB/s 
     |████████████████████████████████| 10.5MB 20.7MB/s 
     |████████████████████████████████| 317kB 56.0MB/s 
     |████████████████████████████████| 6.7MB 23.7MB/s 
     |████████████████████████████████| 748.9MB 20kB/s 
     |████████████████████████████████| 92kB 11.2MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 1.1MB 59.9MB/s 
     |████████████████████████████████| 133kB 61.6MB/s 
     |████████████████████████████████| 7.2MB 65.9MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp36-none-any.whl size=47641 sha256=ead7c2d4dd6f0f6252e7c12762955ee66be77483ff7ea83e304faaf2375d5b97
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1bebc9e946bf5a8cf922e1c86fb6b8a33d9
  Created wheel for pret

# Approach 2: Using CDQA pipeline

In [ ]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

In [ ]:

download_model(model='bert-squad_1.1', dir='./models')

In [ ]:
pd.set_option('max_colwidth',50)
output_df=pd.DataFrame(columns=['paragraphs'])
for index,row in df.iterrows():
  paragraphs=row['text'].split('.')
  length=len(output_df)
  to_append=[paragraphs[:1000]]
  output_df.loc[length]=to_append

output_df.head(1)

,paragraphs
0,[Emergence and Reemergence of Severe Acute Res...


In [ ]:
id_df=pd.DataFrame(df['paper_id'])
id_df=id_df.reset_index(drop=True)
print(id_df.head())
answer_df=pd.concat([id_df,output_df],axis=1)
answer_df=answer_df.rename(columns={'paper_id':'title','text':'paragraphs'})
answer_df.head(1)

                                   paper_id
0  e97c5b77a1087083a18fe6dccdf2f7029cfc8ad7
1  ec7447ab5e1341c5e5818a5dd302caeb51bae7ca
2  ef95bda8107b3bbd8fa5e4e2f9b89e6c2d4ade86
3  e1411444491ad66fb32ede92081dd99eea2b0272
4  73f3955e4f65729a5fcee70f95d85cb4c8aac524


,title,paragraphs
0,e97c5b77a1087083a18fe6dccdf2f7029cfc8ad7,[Emergence and Reemergence of Severe Acute Res...


In [ ]:
!pip install cdqa

In [ ]:
!pip install transformers

# The below cell failed to execute as session crashed due to using up all availaible RAM

In [ ]:
#cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

#cdqa_pipeline.fit_retriever(df=answer_df)

# Approach 3: Using Simple Searching

In [ ]:
queries=[

'What do we know about diagnostics and coronavirus?',
'New advances in diagnosing SARS-COV-2',
'Development of a point-of-care test and rapid bed-side tests',
'Diagnosing SARS-COV-2 with Nucleic-acid based tech',
'Diagnosing SARS-COV-2 with antibodies',
'How does viral load relate to disease presentations and likelihood of a positive diagnostic test?'         
]


In [ ]:
def search_focus(df):
    dfa = df[df['text'].str.contains('covid')]
    dfb = df[df['text'].str.contains('sars')]
    dfc = df[df['text'].str.contains('cov.*\d',regex=True)]
    dfd = df[df['text'].str.contains('ncov')]
    dfe = df[df['text'].str.contains('corona')]
    dfg = df[df['text'].str.contains('mars')]
    dfh = df[df['text'].str.contains('wuhan.*pneumonia',regex=True)]
    dfi = df[df['text'].str.contains('novel.*corona',regex=True)]
    dfj=   df[df['text'].str.contains('wuhan.*virus',regex=True)]
    dfk=  df[df['text'].str.contains('diagnostics',regex=True)]

In [ ]:
import functools
from IPython.core.display import display, HTML
from nltk import PorterStemmer

#tell the system how many sentences are needed
max_sentences=5

tasks = [['diagonistics','diagonistic','positive test','negative test','positive tests','negative tests'],['SARS-CoV-2'],['point-of-care','bed-side'],['nucleic-acid'],['antibody','antibodies'],['viral-load']]


idx=0
for search_words in tasks:
    df_table = pd.DataFrame(columns = ["pub_date","authors","URL","title","excerpt"])
    #search_words=stem_words(search_words)
    # add cov to focus the search the papers and avoid unrelated documents
    #search_words.append("covid")
    # search the dataframe for all the keywords
    df1=df[functools.reduce(lambda a, b: a|b, (df['text'].str.contains(s) for s in search_words))]
    
    display(HTML('<h1>Task Topic: '+queries[idx]+'</h1>'))
    idx+=1
    #display(HTML('# '+str1+' <a></a>'))
    # record how many sentences have been saved for display
    # loop through the result of the dataframe search
    for index, row in df1.iterrows():
        pub_sentence=''
        sentences_used=0
        #break apart the absract to sentence level
        sentences = row['text'].split('. ')
        #loop through the sentences of the text
        for sentence in sentences:
            # missing lets the system know if all the words are in the sentence
            missing=0
            #loop through the words of sentence
            for word in search_words:
                #if keyword missing change missing variable
                if word not in sentence:
                    missing=missing+1
            # after all sentences processed show the sentences not missing keywords limit to max_sentences
            if missing < len(search_words)-1 and sentences_used < max_sentences and len(sentence)<1000 and sentence!='':
                sentence=sentence.capitalize()
                if sentence[len(sentence)-1]!='.':
                    sentence=sentence+'.'
                pub_sentence=pub_sentence+''+sentence
                sentences_used=sentences_used+1
        
        if pub_sentence!='':
            sentence=pub_sentence
            authors=row["authors"].split(" ")
            link=str(row['doi'])
            title=row["title"]
            linka='http://doi.org/'+link
            final_link=linka
            to_append = [row['publish_time'],authors[0]+' et al.',final_link,title,sentence]
            df_length = len(df_table)
            if(len(df_table)<10):
              df_table.loc[df_length] = to_append
            
    #df_table.to_csv(filename,index = False)
    
    pd.set_option('max_colwidth',None)
    display(df_table)
print ("done")

,pub_date,authors,URL,title,excerpt
0,2020-09-06,"Fintzi, et al.",http://doi.org/nan,"Using multiple data streams to estimate and forecast SARS-CoV-2 transmission dynamics, with application to the virus spread in Orange County, California","In addition to estimating transmission and observation model parameters, we report the inferred latent incidence and prevalence and retrospectively evaluate ability of our model to produce short-term forecasts.datawe start with time series of daily tests (positive and negative), case counts (positive tests), and deaths observed over some time period of interest.Next, we develop a model for the number of positive tests (cases), y , observed in time interval (t −1 , t ].Hence, fitting increasingly granular models requires richer surveillance data, e.g., cases, negative tests, and deaths stratified by age and geographical location.For this reason, it is often preferable to work with line lists of positive and negative tests, which often include the geographic and demographic information needed to fit such models.discussionchanges in control/mitigation measures and in human behavior make sars-cov-2 transmission dynamics time-varying."
1,2020-06-03,"Zong, et al.",http://doi.org/nan,Extracting COVID-19 Events from Twitter,"We annotate a corpus of tweets describing positive/negative tests, denied access to testing, deaths due to etc .The top 5 cure or prevention methods that twitter users believe effective are ""social distancing"", ""vitamin c"", ""hydroxychloroquine"", ""chloroquine"" and ""bathroom hygiene"".conclusionsin this paper, we present an annotated corpus of 7,500 tweets for covid-19 events, including positive/negative tests and denied access to testing."
2,2020-04-09,"Friston, et al.",http://doi.org/nan,Dynamic causal modelling of COVID-19,"Note that this is a prior expectation, the effective population size is estimated from the data: the assumption that the effective population size reflects the total population of a country is a hypothesis (that we will test later).initial conditions and population sizethe likelihood or observation modelinitial conditions and population sizethe outcomes considered in figure 2 are new cases (of positive tests and deaths) per day.Interestingly, initially the number of negative tests increases monotonically, while the proportion of positive tests starts to catch up during the peak of the episode."
3,2020-05-16,"Friston, et al.",http://doi.org/nan,Tracking and tracing in the UK: a dynamic causal modelling study,"The orange boxes represent the observable outputs that are generated by this dynamic causal model, in this instance, daily reports of positive tests, daily tests and deaths.Initially, the number of negative tests is about twice the number of positive tests, but this ratio increases as the number of infected people in the population declines.figure 3these predicted trajectories are inferred from the data under the prior assumptions inherent in the model."
4,2020-07-09,"Boukai, et al.",http://doi.org/nan,Bayesian Modeling of COVID-19 Positivity Rate -- the Indiana experience,"Bayesian modeling of covid-19 positivity rate -- the indiana experience in this short technical report we model, within the bayesian framework, the rate of positive tests reported by the the state of indiana, accounting also for the substantial variability (and overdispeartion) in the daily count of the tests performed.While there is substantial (and urgent) effort being made, worldwide, for modeling of the the death rate (or the infection fatality rate, ifr) of covid-19, (see for example basu ), there has been very little attention given to modeling the rate of reported positive tests (often referred to rate of 'positivity').introductionin this report we model, within the bayesian framework, the rate of positive tests reported by the the state of indiana, accounting also for the substantial variability of the daily number test performed.It includes the daily records of the 

,pub_date,authors,URL,title,excerpt


,pub_date,authors,URL,title,excerpt


,pub_date,authors,URL,title,excerpt


,pub_date,authors,URL,title,excerpt
0,2020-08-23,"Samrat, et al.",http://doi.org/10.1016/j.virusres.2020.198141,Prospect of SARS-CoV-2 spike protein: Potential role in vaccine and therapeutic development,"Also, development of a vaccine targeting the s protein is always challenging, as a vaccine based on neutralizing antibodies may lead to antibody-dependent enhancement of secondary virus infection and vaccine-enhanced disease."
1,2020-06-04,"Pickering, et al.",http://doi.org/10.1101/2020.06.02.20120345,Comparative assessment of multiple COVID-19 serological technologies supports continued evaluation of point-of-care lateral flow assays in hospital and community healthcare settings,"However, with no 'gold standard' for antibody testing and an incomplete knowledge of the immunology of covid-19, evaluating tests with the assumption that antibodies 'should' be there, and comparatively to rt-pcr, is problematic.Accordingly, we developed a highly specific semi-quantitative 4 elisa for the detection of anti-spike (s), -s receptor binding domain (rbd) and -n antibodies, and used this to cross-evaluate ten commercial antibody tests (seven lateral flow immunoassays (lfias), one chemiluminescent assay and two elisas) on a collection of 110 serum samples from confirmed rna positive patients, and 50 pre-pandemic samples from march 2019.Weaker antibody responses yielded mixed results from the lfias, with a clear pattern of increasing detection of antibodies seen across all tests with increasing time pos.Association of antibody detection with days pos and severity of diseasesample-by-sample analyses of multiple serological assays showed a trend for increasing detection of antibodies with increasing days pos (up to 30).While this could highlight a characteristic of covid-19 where a subset of patients do not produce a detectable antibody response, we have evidence that in three cases (the two day 10 samples and sixth day 14) where later samples were available (47, 77 and 51 days pos, respectively; the only next available samples) both igm and igg antibodies were detected in several lfias."
2,2020-06-14,La et al.,http://doi.org/10.1016/j.rbmo.2020.06.001,Testing for SARS-CoV-2 (COVID-19): a systematic review and clinical guide to molecular and serological in-vitro diagnostic assays,"The principal difference is that antibody tests require identification of distinct proteins that form the viral coat, with elucidation of which proteins are most divergent from previous coronavirus proteins; then identification of specific antibodies to these proteins that are part of the acquired immune response to viral exposure, and finally testing to ensure that there is limited crossreactivity with antibodies developed to other historical coronaviruses.building an indirect test for sars-cov-2: serological testingwith the previous two coronaviruses a variety of assays encompassing different methodologies were developed including elisa, chemiluminescence, western blot, protein microarray, and immunofluorescence platforms.It is these platforms which are once again being examined for detection of antibodies to sars-cov-2.appraisal of test performanceappropriate thresholds for sensitivity and specificity of an antibody test depend on its purpose and must be considered prior to implementation.In lateral flow assays, a membrane strip is coated with two lines: gold nanoparticle-antibody conjugates are in one line and bind antibodies in the other."
3,2020-07-02,"Dehelean, et al.",http://doi.org/10.3390/jcm9072084,SARS-CoV-2: Repurposed Drugs and Novel Therapeutic Approaches—Insights into Chemical Structure—Biological Activity and Toxicological Screening,"The antibody-dependent enhancement related to viral infection is a paradoxical process characterized by the presence of virus specific antibodies that augment viral entry within host cells and replication of the pathogen, leading to an exacerbation of the disease .The passive immunization by early administration of conv

,pub_date,authors,URL,title,excerpt


done


# Approach 4: Attempting to build sqlite3 database and using SQL queries to get results

In [ ]:
#pip install git+https://github.com/Abhimalya/paperetl.git

In [ ]:
#pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_md-0.2.5.tar.gz

In [ ]:
#!pip uninstall scikit-learn
#!pip install -U scikit-learn==0.21.3

In [ ]:
'''
import csv
import hashlib
import os.path
import shutil
import re
import paperetl

from paperetl import cord19

#from paperetl.cord19.execute import Execute

'''

'\nimport csv\nimport hashlib\nimport os.path\nimport shutil\nimport re\nimport paperetl\n\nfrom paperetl import cord19\n\n#from paperetl.cord19.execute import Execute\n\n'

In [ ]:
import sklearn
#from sklearn.ensemble import RandomForestClassifier

In [ ]:
#os.mkdir("cord19q")

In [ ]:
#shutil.copy("/content/drive/MyDrive/SQLite/attribute", "cord19q")
#shutil.copy("/content/drive/MyDrive/SQLite/entry-dates.csv", "cord19q")

# The below cell failed to execute as session crashed as it used up all avalaible RAM

In [ ]:
#Execute.run("/content/drive/MyDrive/NEW TASK_REVIEW3/jsonfiles10k", "cord19q", "cord19q", "/content/drive/MyDrive/SQLite/entry-dates.csv", False,False)

In [ ]:
#df.head()

# Approach 5: Using Word2Vec along with BERT to improve upon BERT answering

In [ ]:
#Import Packages
import sys
# Set the environment path
sys.path.append("../../")  
import os
from collections import Counter
import math
import numpy as np
import pandas as pd
import gensim
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from scipy.spatial import distance
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


import matplotlib.pyplot as plt
%matplotlib inline


print("System version: {}".format(sys.version))
print("Gensim version: {}".format(gensim.__version__))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
System version: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
Gensim version: 3.8.3


In [ ]:
def prepare_kaggle_df():
    kaggle_df = pd.DataFrame()
   
     
    kaggle_group2 = [ 
                     'Detection Method in Diagnosing SARS-COV-2 with antibodies', 
                     'Sample in Diagnosing SARS-COV-2 with antibodie'
                     'Sample size in Diagnosing SARS-COV-2 with antibodies',       
                     'Measure in evidence in Diagnosing SARS-COV-2 with antibodies',
                     'Speed of assay in Diagnosing SARS-COV-2 with antibodies',
                     'FDA approval in Diagnosing SARS-COV-2 with antibodies'                                
                     
                    ]           
                     
                    
    kaggle_tokens=[nltk.word_tokenize(sents) for sents in kaggle_group2]
    kaggle_df['sentences']=[kaggle_group2]
    kaggle_df['token_sentences']=[kaggle_tokens]
    #print(kaggle_df.sentences[0])
    #print(len(kaggle_df.token_sentences.values))
    #print(len(kaggle_df.token_sentences.values.tolist()))
    return kaggle_df

In [ ]:
number_of_articles  = 100 #sample files with doc2vec similarity threshold 
threshold = 0.4 #threshold for doc2vec cosin similarity 
start_kaggle_index= 0 #start index for particular kaggle query
end_kaggle_index = 5 #index for particular kaggle search query

In [ ]:
def preprocess_data(df,number_of_artciles):
    preprocess_df=preprocses_sentences(df) 
    kaggle_df = prepare_kaggle_df()
    preprocess_df.head()
    kaggle_df.head()
    return preprocess_df,kaggle_df


In [ ]:
def generate_article_df(one_article_df):
    all_articles_relevant_sentences_list = []
    article_df = pd.DataFrame()
    article_df['sentences'] = one_article_df['sentences']
    article_df['token_sentences'] =  one_article_df['token_sentences']
    return article_df

In [ ]:
def transform_kaggle(kaggle_df, start_kaggle_index, end_kaggle_index):
    kaggle_list = [] 
    sent_list = []
    for index in range(len(kaggle_df.token_sentences.iloc[0])):
        kaggle_list.append(kaggle_df.token_sentences.apply(lambda x: x[index]))
    for index in range(len(kaggle_df.sentences.iloc[0])):
        sent_list.append(kaggle_df.sentences.apply(lambda x: x[index]))
    
    df = pd.DataFrame()    
    df = pd.concat(kaggle_list, axis=1,ignore_index=True)
    df2=pd.concat(sent_list,axis=1,ignore_index=True)
    df=df.transpose()
    df2=df2.transpose()
    kaggle_transformed_df = pd.concat([df2,df],axis=1)
    kaggle_transformed_df.columns=['sentences','token_sentences']
    kaggle_transformed_df = kaggle_transformed_df.loc[start_kaggle_index:end_kaggle_index]

    return kaggle_transformed_df

# Doc2Vec

In [ ]:
def prepare_doc2vec_tagged_documents(article_df, transformed_kaggle_df):
    article_sentences = article_df[['sentences']] 
    article_sentences_list = article_sentences.values.flatten().tolist()

 
    kaggle_sentences = transformed_kaggle_df[['sentences']]
    kaggle_sentences_list = kaggle_sentences.values.flatten().tolist()
   
    all_sentences_list = article_sentences_list + kaggle_sentences_list


    corpus = all_sentences_list
    len(corpus)

    # Produce dictionary of sentence to id
    sentence_id = {sent : i for i, sent in enumerate(corpus)} #
    
    # Assign id to sentences
    article_df['qid1'] = article_df['sentences'].apply(lambda row : sentence_id[row])
    transformed_kaggle_df['qid2'] = transformed_kaggle_df['sentences'].apply(lambda row : sentence_id[row])
    
     # Doc2vec requires data as Tagged Documents with the tokenized sentence and the sentence id
    article_df['labeled_tokens_article'] = article_df.apply(lambda x: TaggedDocument(x.token_sentences, str(x.qid1)), axis=1)
    transformed_kaggle_df['labeled_tokens_kaggle'] = transformed_kaggle_df.apply(lambda x: TaggedDocument(x.token_sentences, str(x.qid2)), axis=1)
    
    #Get all Tagged Documents
    labeled_article_sentences = article_df[['labeled_tokens_article']]  
    labeled_article_sentences_list = labeled_article_sentences.values.flatten().tolist()
    #print('num of one article sentences',len(all_sentences))
    labeled_kaggle_sentences = transformed_kaggle_df[['labeled_tokens_kaggle']]
    labeled_kaggle_sentences_list = labeled_kaggle_sentences.values.flatten().tolist()
    labeled_sentences= labeled_article_sentences_list + labeled_kaggle_sentences_list
    
    return labeled_sentences

In [ ]:
def train_doc2vec_model(labeled_sentences):
    model = Doc2Vec(labeled_sentences, dm=1, min_count=1, window=10,negative=5, vector_size=100, epochs=30)
    
    # Train our model for 20 epochs
    for epoch in range(30):
        model.train(labeled_sentences, epochs=model.epochs, total_examples=model.corpus_count)
        
    return model

In [ ]:
def doc2vec_cosine_similarity(article_df,transformed_kaggle_df,doc2vec_model, start_kaggle_index ,end_kaggle_index):
    for index in range(start_kaggle_index,end_kaggle_index):
        name = "task_" + str(index+1)
        kaggle_token = transformed_kaggle_df.loc[index]['token_sentences']
        article_df[name] = article_df.apply(lambda x: doc2vec_model.wv.n_similarity(x.token_sentences,kaggle_token) if len(x.token_sentences) != 0 else 0, axis = 1)
    return article_df


In [ ]:
def doc2vec_per_article(one_article_df,transformed_kaggle_df,threshold, start_kaggle_index ,end_kaggle_index):
    article_df = generate_article_df(one_article_df)
    labeled_sentences = prepare_doc2vec_tagged_documents(article_df,transformed_kaggle_df)
    doc2vec_model = train_doc2vec_model(labeled_sentences)
    transformed_article_df=doc2vec_cosine_similarity(article_df,transformed_kaggle_df,doc2vec_model,start_kaggle_index ,end_kaggle_index)
    relevant_sentences_one_article= find_top_relevant_sentences(transformed_article_df,transformed_kaggle_df,threshold, start_kaggle_index ,end_kaggle_index) #put the threshold 
    return relevant_sentences_one_article

In [ ]:
def word2vec_per_article(one_article_df,transformed_kaggle_df,threshold,start_kaggle_index ,end_kaggle_index):
    article_df = generate_article_df(one_article_df)
    sentence_embeddings_list,kaggle_embeddings_list= generate_word_embeddings(article_df,transformed_kaggle_df)
    transformed_article_df=generate_cosin_similarity(article_df,sentence_embeddings_list,kaggle_embeddings_list)
    relevant_sentences_one_article = find_top_relevant_sentences(transformed_article_df,transformed_kaggle_df,threshold, start_kaggle_index ,end_kaggle_index)
    return relevant_sentences_one_article
    

In [ ]:
def find_top_relevant_sentences(article_df,kaggle_transformed_df,threshold,start_kaggle_index ,end_kaggle_index):
    relevant_sentences_df = pd.DataFrame()
    deduplicated_relevante_top_sentences = pd.DataFrame()
    final_relevant_sentences_list = []
    tmp_final=pd.DataFrame(columns={'sentences','score'})
    for index in range(start_kaggle_index,end_kaggle_index): 
        name = "task_" + str(index+1)
        tmp_df=article_df.sort_values(name, inplace = False, ascending=False).head(5) #sorting and select the top 5 sentences
        tmp2_df = tmp_df[['sentences',name]]
        tmp2_df = tmp2_df.rename(columns={name : "score"})
        tmp_final=tmp_final.append(tmp2_df, ignore_index=True) #concatinate all the sentences and score and scores
    relevant_sentences_df=tmp_final.sort_values('score',inplace = False, ascending = False) #sorting
    deduplicated_relevante_top_sentences = relevant_sentences_df.drop_duplicates('sentences',keep='first') #drop duplicates
    deduplicated_relevante_top_sentences['score'] = pd.to_numeric(deduplicated_relevante_top_sentences['score'],errors='coerce')
    final_relevante_sentences=deduplicated_relevante_top_sentences[deduplicated_relevante_top_sentences.score > threshold]
    return final_relevante_sentences 


In [ ]:
def find_relevant_sentences_all_articles(preprocess_df,kaggle_df, threshold, start_kaggle_index,end_kaggle_index):
    all_relevant_sentences_list = []
    transformed_kaggle_df = transform_kaggle(kaggle_df,start_kaggle_index,end_kaggle_index)
    for index in range(len(preprocess_df)):
        one_article_relevant_sentences_list = []
        one_article_df=preprocess_df.iloc[index]
        #relevant_sentences_one_article=doc2vec_per_article(one_article_df,transformed_kaggle_df,threshold, start_kaggle_index ,end_kaggle_index)
        relevant_sentences_one_article = word2vec_per_article(one_article_df,transformed_kaggle_df,threshold,start_kaggle_index ,end_kaggle_index) #using word2vec instead of doc2vec
        for i in range(len(relevant_sentences_one_article)):
            one_article_relevant_sentences_list.append(relevant_sentences_one_article.iloc[i]['sentences'])
        all_relevant_sentences_list.append(one_article_relevant_sentences_list)
    preprocess_df['extracted_relevant_sentences_doc2word_group5'] = all_relevant_sentences_list
    return preprocess_df

# Word2Vec

In [ ]:
pip install -U gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


These settings are important for processing the articles for different methods. 


# Preprocess Data

In [ ]:
def apply_all_sent_token(doc):
    list_all_tokens = []
    all_sent = [nltk.word_tokenize(sents) for sents in doc]
    for one_sent in all_sent:
        filtered_tokens = []
        filtered_tokens =[words for words in one_sent if words not in stop_words]
        custom_removed_tokens = [words for words in filtered_tokens if words not in ['.','shown','fig.','figure','fig']]
        custom_removed_tokens = [words for words in custom_removed_tokens if len(words) > 2]
        list_all_tokens.append(custom_removed_tokens)
    return list_all_tokens

In [ ]:
def split_sentences(article):
    import nltk.data
    import re
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    list_df = []
    marked_sentences = '***'.join(tokenizer.tokenize(article))
    marked_sentences = re.sub(r"(\[[0-9]*\])",'',marked_sentences) #remove the citations 
    marked_sentences = re.sub(r"(\([0-9]*\))",'',marked_sentences)
    list_df  = marked_sentences.split('***')
    return list_df

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
word2vec_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NEW TASK_REVIEW3/GoogleNews-vectors-negative300.bin', binary =True)

In [ ]:
def generate_word_embeddings(transformed_article_df,transformed_kaggle_df):   
    all_sentences = transformed_article_df[['token_sentences']]
    all_sentences_list = all_sentences.values.flatten().tolist()
    #print(len(all_sentences_list))
    kaggle_sentences = transformed_kaggle_df[['token_sentences']]
    #print(kaggle_sentences)
    kaggle_sentenes_list = kaggle_sentences.values.flatten().tolist()
    #print(len(kaggle_sentenes_list))
    all_sentences = all_sentences_list + kaggle_sentenes_list
    #print(len(all_sentences))
    document_frequency_dict, num_documnets = get_document_frequency(all_sentences)
    sentence_embeddings_list = average_word_embedding_cosine_similarity(transformed_article_df, word2vec_model,document_frequency_dict,num_documnets)
    kaggle_embeddings_list = average_word_embedding_cosine_similarity(transformed_kaggle_df, word2vec_model,document_frequency_dict,num_documnets)
    #print('sentence_embeddings ' ,len(sentence_embeddings_list))
    #print(len(kaggle_embeddings_list))
    return sentence_embeddings_list,kaggle_embeddings_list

In [ ]:
def generate_cosin_similarity(transformed_article_df,sentence_embeddings_list,kaggle_embeddings_list):
    for index in range(len(kaggle_embeddings_list)): #len(kaggle_embeddings_list)
        cosine_similarity = []
        name = "task_" + str(index+1)
        for index2 in range(len(sentence_embeddings_list)):
            one_sentence_embeddings = sentence_embeddings_list[index2]
            kaggle_embeddings = kaggle_embeddings_list[index]
            if sum(one_sentence_embeddings) != 0 and sum(kaggle_embeddings)!= 0:
                #print(calculate_modified_cosine_similarity(one_sentence_embeddings,kaggle_embeddings))
                cosine_similarity.append(calculate_cosine_similarity(one_sentence_embeddings,kaggle_embeddings))
            else: 
                #print(0)
                cosine_similarity.append(0)
        #print(len(cosine_similarity))
        transformed_article_df[name]= cosine_similarity 
    #cosine_df[name]=pd.Series(cosine_similarity) 
    return transformed_article_df

In [ ]:
def get_document_frequency(all_sentences_list):
    """Iterate through all sentences in dataframe and create a dictionary 
    mapping tokens to the number of sentences in our corpus they appear in
    
    Args:
        df (pandas dataframe): dataframe of sentence pairs with their similarity scores
        
    Returns:
        document_frequency_dict (dictionary): mapping from tokens to number of sentences they appear in
        n (int): number of sentences in the corpus
    """
    document_frequency_dict = {}
    all_sentences = []
    all_sentences = all_sentences_list     
    n = len(all_sentences)

    for s in all_sentences:
        for token in set(s):
            document_frequency_dict[token] = document_frequency_dict.get(token, 0) + 1

    return document_frequency_dict, n

In [ ]:
def average_word_embedding_cosine_similarity(df, embedding_model, document_frequencies,num_documnets, rm_stopwords=False):
    """Calculate the cosine similarity between TF-IDF weighted averaged embeddings
    
    Args:
        df (pandas dataframe): dataframe as provided by the nlp_utils
        embedding_model: word embedding model
        rm_stopwords (bool): whether to remove stop words (True) or not (False)
    
    Returns:
        list: predicted values for sentence similarity of test set examples
    """
   
    sentence_embedding = df.apply(lambda x: average_sentence_embedding(x.token_sentences, embedding_model,document_frequencies, num_documnets), axis=1)
     
    return sentence_embedding.tolist()

In [ ]:
def calculate_cosine_similarity(embedding1, embedding2):
    """Calculate cosine similarity between two embedding vectors
    
    Args:
        embedding1 (list): embedding for the first sentence
        embedding2 (list): embedding for the second sentence
    
    Returns:
        list: cosine similarity value between the two embeddings
    """
    # distance.cosine calculates cosine DISTANCE, so we need to
    # return 1 - distance to get cosine similarity
    cosine_similarity = 1 - distance.cosine(embedding1, embedding2)
    return cosine_similarity

In [ ]:
def average_sentence_embedding(tokens, embedding_model,document_frequencies, num_documents):
    """Calculate TF-IDF weighted average embedding for a sentence
    
    Args:
        tokens (list): list of tokens in a sentence
        embedding_model: model to use for word embedding (word2vec, glove, fastText, etc.)
    
    Returns:
        list: vector representing the sentence
    """
    # Throw away tokens that are not in the embedding model
    tokens = [i for i in tokens if i in embedding_model]

    if len(tokens) == 0:
        return []

    # We will weight by TF-IDF. The TF part is calculated by:
    # (# of times term appears / total terms in sentence)
    count = Counter(tokens)
    token_list = list(count)
    term_frequency = [count[i] / len(tokens) for i in token_list]

    #print(num_documents)
    # Now for the IDF part: LOG(# documents / # documents with term in it)
    inv_doc_frequency = [
        math.log(num_documents / (document_frequencies.get(i, 0) + 1)) for i in count
    ]

    # Put the TF-IDF together and produce the weighted average of vector embeddings
    word_embeddings = [embedding_model[token] for token in token_list]
    weights = [term_frequency[i] * inv_doc_frequency[i] for i in range(len(token_list))]
    return list(np.average(word_embeddings, weights=weights, axis=0))

## Run Methods

In [ ]:
df.head()

,paper_id,title,publish_time,subset,doi,text,language,tokenized_text
1134,e97c5b77a1087083a18fe6dccdf2f7029cfc8ad7,Emergence and Reemergence of Severe Acute Resp...,2020-04-30,json,10.1007/978-981-15-4814-7_13,Emergence and Reemergence of Severe Acute Resp...,en,"[Emergence, and, Reemergence, of, Severe, Acut..."
1145,ec7447ab5e1341c5e5818a5dd302caeb51bae7ca,From Diversity to Coordination: A European App...,2020-04-16,json,10.1017/err.2020.36,From Diversity to Coordination: A European App...,en,"[From, Diversity, to, Coordination, :, A, Euro..."
1324,ef95bda8107b3bbd8fa5e4e2f9b89e6c2d4ade86,Policing During the Time of Corona: The Indian...,2020-05-31,json,10.1093/police/paaa024,Policing During the Time of Corona: The Indian...,en,"[Policing, During, the, Time, of, Corona, :, T..."
1456,e1411444491ad66fb32ede92081dd99eea2b0272,"Fighting impunity in hate crime — history, eth...",2020-08-20,json,10.1007/s41020-020-00118-1,"Fighting impunity in hate crime — history, eth...",en,"[Fighting, impunity, in, hate, crime, —, histo..."
1491,73f3955e4f65729a5fcee70f95d85cb4c8aac524,Why do so many trials of vitamin D supplementa...,2020-08-11,json,10.1530/ec-20-0274,Why do so many trials of vitamin D supplementa...,en,"[Why, do, so, many, trials, of, vitamin, D, su..."


In [ ]:
preprocess_df=df
kaggle_df = prepare_kaggle_df()
preprocess_df=preprocess_df.rename(columns={'text':'body_text'})
preprocess_df.head()


,paper_id,title,publish_time,subset,doi,body_text,language,tokenized_text
1134,e97c5b77a1087083a18fe6dccdf2f7029cfc8ad7,Emergence and Reemergence of Severe Acute Resp...,2020-04-30,json,10.1007/978-981-15-4814-7_13,Emergence and Reemergence of Severe Acute Resp...,en,"[Emergence, and, Reemergence, of, Severe, Acut..."
1145,ec7447ab5e1341c5e5818a5dd302caeb51bae7ca,From Diversity to Coordination: A European App...,2020-04-16,json,10.1017/err.2020.36,From Diversity to Coordination: A European App...,en,"[From, Diversity, to, Coordination, :, A, Euro..."
1324,ef95bda8107b3bbd8fa5e4e2f9b89e6c2d4ade86,Policing During the Time of Corona: The Indian...,2020-05-31,json,10.1093/police/paaa024,Policing During the Time of Corona: The Indian...,en,"[Policing, During, the, Time, of, Corona, :, T..."
1456,e1411444491ad66fb32ede92081dd99eea2b0272,"Fighting impunity in hate crime — history, eth...",2020-08-20,json,10.1007/s41020-020-00118-1,"Fighting impunity in hate crime — history, eth...",en,"[Fighting, impunity, in, hate, crime, —, histo..."
1491,73f3955e4f65729a5fcee70f95d85cb4c8aac524,Why do so many trials of vitamin D supplementa...,2020-08-11,json,10.1530/ec-20-0274,Why do so many trials of vitamin D supplementa...,en,"[Why, do, so, many, trials, of, vitamin, D, su..."


In [ ]:
def preprocses_sentences(df):

    columns=list(df.columns)
    columns.append('sentences')
    columns.append('token_sentences')
    preprocess_df=pd.DataFrame(columns=columns)
    df_sent = df['text'].apply(lambda x: split_sentences(x)) 
    df_lower = df_sent.apply(lambda doc: [sent.lower() if type(sent) == str else sent for sent in doc])
    df_clean = df_lower.apply(lambda doc: [re.sub(r"[^.a-z]",' ',sent) for sent in doc]) #remove punctation
    df_token = df_clean.apply(lambda doc:apply_all_sent_token(doc)) #tokenize all the sentences 
    preprocess_df=pd.concat([df,df_sent],axis = 1)
    preprocess_df=pd.concat([preprocess_df, df_token],axis=1)
    preprocess_df.columns=columns
    return preprocess_df

In [ ]:
columns=list(df.columns)
columns.append('sentences')
columns.append('token_sentences')
mydf=pd.DataFrame(columns=columns)
mydf.columns

Index(['paper_id', 'title', 'publish_time', 'subset', 'doi', 'text',
       'language', 'tokenized_text', 'sentences', 'token_sentences'],
      dtype='object')

In [ ]:
def apply_all_sent_token(doc):
    list_all_tokens = []
    all_sent = [nltk.word_tokenize(sents) for sents in doc]
    for one_sent in all_sent:
        filtered_tokens = []
        filtered_tokens =[words for words in one_sent if words not in stop_words]
        custom_removed_tokens = [words for words in filtered_tokens if words not in ['.','shown','fig.','figure','fig']]
        custom_removed_tokens = [words for words in custom_removed_tokens if len(words) > 2]
        list_all_tokens.append(custom_removed_tokens)
    return list_all_tokens

In [ ]:
def split_sentences(article):
    import nltk.data
    import re
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    list_df = []
    marked_sentences = '***'.join(tokenizer.tokenize(article))
    marked_sentences = re.sub(r"(\[[0-9]*\])",'',marked_sentences) #remove the citations 
    marked_sentences = re.sub(r"(\([0-9]*\))",'',marked_sentences)
    list_df  = marked_sentences.split('***')
    return list_df

In [ ]:
preprocess_df=preprocses_sentences(df)
preprocess_df.head()
#preprocess_df.columns

,paper_id,title,publish_time,subset,doi,text,language,tokenized_text,sentences,token_sentences
1134,e97c5b77a1087083a18fe6dccdf2f7029cfc8ad7,Emergence and Reemergence of Severe Acute Resp...,2020-04-30,json,10.1007/978-981-15-4814-7_13,Emergence and Reemergence of Severe Acute Resp...,en,"[Emergence, and, Reemergence, of, Severe, Acut...",[Emergence and Reemergence of Severe Acute Res...,"[[emergence, reemergence, severe, acute, respi..."
1145,ec7447ab5e1341c5e5818a5dd302caeb51bae7ca,From Diversity to Coordination: A European App...,2020-04-16,json,10.1017/err.2020.36,From Diversity to Coordination: A European App...,en,"[From, Diversity, to, Coordination, :, A, Euro...",[From Diversity to Coordination: A European Ap...,"[[diversity, coordination, european, approach,..."
1324,ef95bda8107b3bbd8fa5e4e2f9b89e6c2d4ade86,Policing During the Time of Corona: The Indian...,2020-05-31,json,10.1093/police/paaa024,Policing During the Time of Corona: The Indian...,en,"[Policing, During, the, Time, of, Corona, :, T...",[Policing During the Time of Corona: The India...,"[[policing, time, corona, indian, context, acc..."
1456,e1411444491ad66fb32ede92081dd99eea2b0272,"Fighting impunity in hate crime — history, eth...",2020-08-20,json,10.1007/s41020-020-00118-1,"Fighting impunity in hate crime — history, eth...",en,"[Fighting, impunity, in, hate, crime, —, histo...","[Fighting impunity in hate crime — history, et...","[[fighting, impunity, hate, crime, history, et..."
1491,73f3955e4f65729a5fcee70f95d85cb4c8aac524,Why do so many trials of vitamin D supplementa...,2020-08-11,json,10.1530/ec-20-0274,Why do so many trials of vitamin D supplementa...,en,"[Why, do, so, many, trials, of, vitamin, D, su...",[Why do so many trials of vitamin D supplement...,"[[many, trials, vitamin, supplementation, fail..."


In [ ]:
final_preprocess_df = find_relevant_sentences_all_articles(preprocess_df,kaggle_df,threshold,start_kaggle_index ,end_kaggle_index)
final_preprocess_df.head()

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A val

,paper_id,title,publish_time,subset,doi,text,language,tokenized_text,sentences,token_sentences,extracted_relevant_sentences_doc2word_group5
1134,e97c5b77a1087083a18fe6dccdf2f7029cfc8ad7,Emergence and Reemergence of Severe Acute Resp...,2020-04-30,json,10.1007/978-981-15-4814-7_13,Emergence and Reemergence of Severe Acute Resp...,en,"[Emergence, and, Reemergence, of, Severe, Acut...",[Emergence and Reemergence of Severe Acute Res...,"[[emergence, reemergence, severe, acute, respi...",[The remaining part of viral genome encodes sp...
1145,ec7447ab5e1341c5e5818a5dd302caeb51bae7ca,From Diversity to Coordination: A European App...,2020-04-16,json,10.1017/err.2020.36,From Diversity to Coordination: A European App...,en,"[From, Diversity, to, Coordination, :, A, Euro...",[From Diversity to Coordination: A European Ap...,"[[diversity, coordination, european, approach,...","[COVID-19, for example, is an ""invisible risk""..."
1324,ef95bda8107b3bbd8fa5e4e2f9b89e6c2d4ade86,Policing During the Time of Corona: The Indian...,2020-05-31,json,10.1093/police/paaa024,Policing During the Time of Corona: The Indian...,en,"[Policing, During, the, Time, of, Corona, :, T...",[Policing During the Time of Corona: The India...,"[[policing, time, corona, indian, context, acc...","[In order to establish the above hypotheses, t..."
1456,e1411444491ad66fb32ede92081dd99eea2b0272,"Fighting impunity in hate crime — history, eth...",2020-08-20,json,10.1007/s41020-020-00118-1,"Fighting impunity in hate crime — history, eth...",en,"[Fighting, impunity, in, hate, crime, —, histo...","[Fighting impunity in hate crime — history, et...","[[fighting, impunity, hate, crime, history, et...","[According to it, Muslims actually want to eng..."
1491,73f3955e4f65729a5fcee70f95d85cb4c8aac524,Why do so many trials of vitamin D supplementa...,2020-08-11,json,10.1530/ec-20-0274,Why do so many trials of vitamin D supplementa...,en,"[Why, do, so, many, trials, of, vitamin, D, su...",[Why do so many trials of vitamin D supplement...,"[[many, trials, vitamin, supplementation, fail...",[Since the 25(OH)D assays had been carried out...


# BERT Pre-trained Question-Answer Model


In [ ]:
def BERT_Qquestion_Answer(question,text):
    
    from transformers import BertTokenizer, BertForQuestionAnswering
    import torch
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased-finetuned-mrpc')
    model = BertForQuestionAnswering.from_pretrained('bert-base-cased-finetuned-mrpc')
    
    input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
    import re
    encoding = tokenizer.encode_plus(
                    input_text,                      # Sentence to encode.
                    max_length = 512,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,  # Construct attn. masks.
                    truncation=True
    
    )

    input_ids, token_type_ids = encoding["input_ids"], encoding["token_type_ids"]
    start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]),return_dict=False)
 
    #Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
         #If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

        
    return answer


In [ ]:
new_df  = final_preprocess_df.sample(frac=0.01,random_state=42)
new_df.count()
#print(len(new_df))
new_df

,paper_id,title,publish_time,subset,doi,text,language,tokenized_text,sentences,token_sentences,extracted_relevant_sentences_doc2word_group5
3622,70e4c0204b68d7c3516c6ad6713704f2ec20bc27,High prevalence of obesity in severe acute res...,2020-04-09,json,10.1002/oby.22831,High prevalence of obesity in severe acute res...,en,"[High, prevalence, of, obesity, in, severe, ac...",[High prevalence of obesity in severe acute re...,"[[high, prevalence, obesity, severe, acute, re...",[Throat swab samples were obtained from all pa...
3426,e678d71eee9e7602e429a78be1c8ccf666ef9370,Predictors of coronavirus disease 19 (COVID-19...,2020-08-08,json,10.1007/s10140-020-01833-x,Predictors of coronavirus disease 19 (COVID-19...,en,"[Predictors, of, coronavirus, disease, 19, (, ...",[Predictors of coronavirus disease 19 (COVID-1...,"[[predictors, coronavirus, disease, covid, pne...",[First reports have shown that computed tomogr...
6997,f8b5ff06f641d2b2f55aec0bd245282707c625a2,Modelling the downhill of the Sars-Cov-2 in It...,2020-07-01,json,10.1016/j.chaos.2020.110064,Modelling the downhill of the Sars-Cov-2 in It...,en,"[Modelling, the, downhill, of, the, Sars-Cov-2...",[Modelling the downhill of the Sars-Cov-2 in I...,"[[modelling, downhill, sars, cov, italy, unive...",[Compared to the previous section we observe a...
4725,72b3b9a9cad3344fc3b2acb3758176feacec0db6,Bibliometric analysis of scientific publicatio...,2020-07-28,json,10.1186/s12992-020-00602-2,Bibliometric analysis of scientific publicatio...,en,"[Bibliometric, analysis, of, scientific, publi...",[Bibliometric analysis of scientific publicati...,"[[bibliometric, analysis, scientific, publicat...",[Substance use disorders have been associated ...
3203,edddb2be63dda9f1aad841321aaca1d46e503ef7,Use of Tracheostomy During the COVID-19 Pandem...,2020-06-06,json,10.1016/j.chest.2020.05.571,Use of Tracheostomy During the COVID-19 Pandem...,en,"[Use, of, Tracheostomy, During, the, COVID-19,...",[Use of Tracheostomy During the COVID-19 Pande...,"[[use, tracheostomy, covid, pandemic, chest, a...",[In another study on critically ill 2 patients...
5536,ef07fbe4e6680ae594a4b438a1e3cacc10a3fd86,COVID-19 and Stroke: incidence and etiological...,2020-08-05,json,10.1016/j.jstrokecerebrovasdis.2020.105225,COVID-19 and Stroke: incidence and etiological...,en,"[COVID-19, and, Stroke, :, incidence, and, eti...",[COVID-19 and Stroke: incidence and etiologica...,"[[covid, stroke, incidence, etiological, descr...","[At the end of the procedure, recanalization w..."
332,f8ccf687bf28dcd4c776783692ed8ea2b2702cf0,Miller Fisher syndrome with bilateral vocal co...,2020-02-18,json,10.1186/s13256-020-2357-4,Miller Fisher syndrome with bilateral vocal co...,en,"[Miller, Fisher, syndrome, with, bilateral, vo...",[Miller Fisher syndrome with bilateral vocal c...,"[[miller, fisher, syndrome, bilateral, vocal, ...",[Anti-GQ1b antibodies develop following an inf...
2500,ef297364cc36822b9ffcb378b054de2207e7035c,Therapeutic Strategies in the Development of A...,2020-08-18,json,10.1007/s12035-020-02074-2,Therapeutic Strategies in the Development of A...,en,"[Therapeutic, Strategies, in, the, Development...",[Therapeutic Strategies in the Development of ...,"[[therapeutic, strategies, development, anti, ...",[Tocilizumab is a human monoclonal anti-IL-6 r...
3007,e79e68ebbbd5d1dd8bea17c85910a89385030346,Role of Monocytes/Macrophages in Covid-19 Path...,2020-07-22,json,10.2147/idr.s258639,Role of Monocytes/Macrophages in Covid-19 Path...,en,"[Role, of, Monocytes/Macrophages, in, Covid-19...",[Role of Monocytes/Macrophages in Covid-19 Pat...,"[[role, monocytes, macrophages, covid, pathoge...","[Lenzilumab (KB003), is a humanized monoclonal..."
5230,71ff5527563ff744414e53b4d315db61eb3bfc8f,Initial Clinical Impressions of the Critical C...,2020-04-20,json,10.1213/ane.0000000000004830,Initial Clinical Impressions of the Critical C...,en,"[Initial, Clinical, Impressions, of, the, Crit...",[Initial Clinical Impressions of the Critical 

In [ ]:
'''
new_df  = final_preprocess_df.sample(frac=0.01,random_state=42)
new_df.count() #400 files for training
sentence_list = preprocess_df.extracted_relevant_sentences_doc2word_group5.apply(lambda x:x)
question_list = ['what is the study type?','what detection method is used?','what is the sample size?','what sample is obtained?',
                 'what are the measures of evidence?','what is the speed of assay?','Is it FDA approval?']

all_answer_list = []
for qindex in range(0,len(question_list)):
    answer_list = list()
    for index in range(len(sentence_list)):
        mystr="".join(list(sentence_list)[index])
        answer=BERT_Qquestion_Answer(question_list[qindex],mystr)
        #print('answer',answer)
        answer_list.append(answer)
    all_answer_list.append(answer_list)
'''

'\nnew_df  = final_preprocess_df.sample(frac=0.01,random_state=42)\nnew_df.count() #400 files for training\nsentence_list = preprocess_df.extracted_relevant_sentences_doc2word_group5.apply(lambda x:x)\nquestion_list = [\'what is the study type?\',\'what detection method is used?\',\'what is the sample size?\',\'what sample is obtained?\',\n                 \'what are the measures of evidence?\',\'what is the speed of assay?\',\'Is it FDA approval?\']\n\nall_answer_list = []\nfor qindex in range(0,len(question_list)):\n    answer_list = list()\n    for index in range(len(sentence_list)):\n        mystr="".join(list(sentence_list)[index])\n        answer=BERT_Qquestion_Answer(question_list[qindex],mystr)\n        #print(\'answer\',answer)\n        answer_list.append(answer)\n    all_answer_list.append(answer_list)\n'

# The above cell failed to execute. The session kept on crashing due to using up all availaible RAM

In [ ]:
#new_df['study_type'] = all_answer_list[0]
#new_df['method'] = all_answer_list[1]
#new_df['sample_size'] = all_answer_list[2]
#new_df['Sample_obtained'] = all_answer_list[3]
#new_df['measure'] = all_answer_list[4]
#new_df['assay'] = all_answer_list[5]
#new_df['FDA'] = all_answer_list[6]

# Approach 6 Using BioBERT model and covid_bert-based model from deepset.ai.This approach is similar to approach 1

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

model = AutoModelForMaskedLM.from_pretrained("deepset/covid_bert_base")

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer2 = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

model2 = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("clagator/biobert_squad2_cased")

model = AutoModelForQuestionAnswering.from_pretrained("clagator/biobert_squad2_cased")


# Using biobert-squad2-cased

In [ ]:
def answer_question(question,text):
    inputs = tokenizer.batch_encode_plus(
          [(question, text) for text in mystr_list] , add_special_tokens=True, return_tensors='tf',
          max_length=511, truncation_strategy='only_second', pad_to_max_length=True,truncation=True)

    input_ids = inputs['input_ids'].numpy()
    print(type(input_ids))


    start_scores, end_scores = model(inputs)

    #start_scores= model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 #token_type_ids=torch.tensor([segment_ids]),return_dict=False) # The segment IDs to differentiate question from answer_text

    answer_start = tf.argmax(answer_start_scores, axis=1).numpy()  # Get the most likely beginning of each answer with the argmax of the score
    answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()  # Get the most likely end of each answer with the argmax of the score
    
    #print(answer_end)

    # Combine the tokens in the answer and print it out.
    answer = ' '.join(tokens[answer_start:answer_end+1])

    #print(len(answer))
    #if(len(answer))
    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    #print('Answer: "' + answer + '"') 
    #print (answer)
    if(len(answer)<10):
      answer="Appropriate answer not found"
    return answer

View the answers for our chosen questions below:

# The above cell failed to execute. There are some errors in the 'modelling.py' file of the model.